**Source of the materials**: Biopython cookbook (adapted)
<font color='red'>Status: Draft</font>


**Fonte dos materiais** : livro de receitas Biopython (adaptado)

 &lt;font color = &#39;red&#39;&gt; Status: Rascunho &lt;/font&gt;


Cluster analysis
================

Cluster analysis is the grouping of items into clusters based on the
similarity of the items to each other. In bioinformatics, clustering is
widely used in gene expression data analysis to find groups of genes
with similar gene expression profiles. This may identify functionally
related genes, as well as suggest the function of presently unknown
genes.

The Biopython module `Bio.Cluster` provides commonly used clustering
algorithms and was designed with the application to gene expression data
in mind. However, this module can also be used for cluster analysis of
other types of data. `Bio.Cluster` and the underlying C Clustering
Library is described by De Hoon *et al.* @dehoon2004.

The following four clustering approaches are implemented in
`Bio.Cluster`:

-   Hierarchical clustering (pairwise centroid-, single-, complete-, and
    average-linkage);

-   $k$-means, $k$-medians, and $k$-medoids clustering;

-   Self-Organizing Maps;

-   Principal Component Analysis.

### Data representation {#data-representation .unnumbered}

The data to be clustered are represented by a $n \times m$ Numerical
Python array `data`. Within the context of gene expression data
clustering, typically the rows correspond to different genes whereas the
columns correspond to different experimental conditions. The clustering
algorithms in `Bio.Cluster` can be applied both to rows (genes) and to
columns (experiments).

### Missing values {#missing-values .unnumbered}

Often in microarray experiments, some of the data values are missing,
which is indicated by an additional $n \times m$ Numerical Python
integer array `mask`. If `mask[i,j]==0`, then `data[i,j]` is missing and
is ignored in the analysis.

### Random number generator {#random-number-generator .unnumbered}

The $k$-means/medians/medoids clustering algorithms and Self-Organizing
Maps (SOMs) include the use of a random number generator. The uniform
random number generator in `Bio.Cluster` is based on the algorithm by
L’Ecuyer @lecuyer1988, while random numbers following the binomial
distribution are generated using the BTPE algorithm by Kachitvichyanukul
and Schmeiser @kachitvichyanukul1988. The random number generator is
initialized automatically during its first call. As this random number
generator uses a combination of two multiplicative linear congruential
generators, two (integer) seeds are needed for initialization, for which
we use the system-supplied random number generator `rand` (in the C
standard library). We initialize this generator by calling `srand` with
the epoch time in seconds, and use the first two random numbers
generated by `rand` as seeds for the uniform random number generator in
`Bio.Cluster`.

Distance functions {#sec:distancefunctions}
------------------

In order to cluster items into groups based on their similarity, we
should first define what exactly we mean by *similar*. `Bio.Cluster`
provides eight distance functions, indicated by a single character, to
measure similarity, or conversely, distance:

-   `'e'`: Euclidean distance;

-   `'b'`: City-block distance.

-   `'c'`: Pearson correlation coefficient;

-   `'a'`: Absolute value of the Pearson correlation coefficient;

-   `'u'`: Uncentered Pearson correlation (equivalent to the cosine of
    the angle between two data vectors);

-   `'x'`: Absolute uncentered Pearson correlation;

-   `'s'`: Spearman’s rank correlation;

-   `'k'`: Kendall’s $\tau$.

The first two are true distance functions that satisfy the triangle
inequality:
$$d\left(\underline{u},\underline{v}\right) \leq d\left(\underline{u},\underline{w}\right) + d\left(\underline{w},\underline{v}\right) \textrm{ for all } \underline{u}, \underline{v}, \underline{w},$$
and are therefore refered to as *metrics*. In everyday language, this
means that the shortest distance between two points is a straight line.

The remaining six distance measures are related to the correlation
coefficient, where the distance $d$ is defined in terms of the
correlation $r$ by $d=1-r$. Note that these distance functions are
*semi-metrics* that do not satisfy the triangle inequality. For example,
for $$\underline{u}=\left(1,0,-1\right);$$
$$\underline{v}=\left(1,1,0\right);$$
$$\underline{w}=\left(0,1,1\right);$$ we find a Pearson distance
$d\left(\underline{u},\underline{w}\right) = 1.8660$, while
$d\left(\underline{u},\underline{v}\right)+d\left(\underline{v},\underline{w}\right) = 1.6340$.

### Euclidean distance {#euclidean-distance .unnumbered}

In `Bio.Cluster`, we define the Euclidean distance as
$$d = {1 \over n} \sum_{i=1}^{n} \left(x_i-y_i\right)^{2}.$$ Only those
terms are included in the summation for which both $x_i$ and $y_i$ are
present, and the denominator $n$ is chosen accordingly. As the
expression data $x_i$ and $y_i$ are subtracted directly from each other,
we should make sure that the expression data are properly normalized
when using the Euclidean distance.

### City-block distance {#city-block-distance .unnumbered}

The city-block distance, alternatively known as the Manhattan distance,
is related to the Euclidean distance. Whereas the Euclidean distance
corresponds to the length of the shortest path between two points, the
city-block distance is the sum of distances along each dimension. As
gene expression data tend to have missing values, in `Bio.Cluster` we
define the city-block distance as the sum of distances divided by the
number of dimensions:
$$d = {1 \over n} \sum_{i=1}^n \left|x_i-y_i\right|.$$ This is equal to
the distance you would have to walk between two points in a city, where
you have to walk along city blocks. As for the Euclidean distance, the
expression data are subtracted directly from each other, and we should
therefore make sure that they are properly normalized.

### The Pearson correlation coefficient {#the-pearson-correlation-coefficient .unnumbered}

The Pearson correlation coefficient is defined as
$$r = \frac{1}{n} \sum_{i=1}^n \left( \frac{x_i -\bar{x}}{\sigma_x} \right) \left(\frac{y_i -\bar{y}}{\sigma_y} \right),$$
in which $\bar{x}, \bar{y}$ are the sample mean of $x$ and $y$
respectively, and $\sigma_x, \sigma_y$ are the sample standard deviation
of $x$ and $y$. The Pearson correlation coefficient is a measure for how
well a straight line can be fitted to a scatterplot of $x$ and $y$. If
all the points in the scatterplot lie on a straight line, the Pearson
correlation coefficient is either +1 or -1, depending on whether the
slope of line is positive or negative. If the Pearson correlation
coefficient is equal to zero, there is no correlation between $x$ and
$y$.

The *Pearson distance* is then defined as
$$d_{\textrm{P}} \equiv 1 - r.$$ As the Pearson correlation coefficient
lies between -1 and 1, the Pearson distance lies between 0 and 2.

### Absolute Pearson correlation {#absolute-pearson-correlation .unnumbered}

By taking the absolute value of the Pearson correlation, we find a
number between 0 and 1. If the absolute value is 1, all the points in
the scatter plot lie on a straight line with either a positive or a
negative slope. If the absolute value is equal to zero, there is no
correlation between $x$ and $y$.

The corresponding distance is defined as
$$d_{\textrm A} \equiv 1 - \left|r\right|,$$ where $r$ is the Pearson
correlation coefficient. As the absolute value of the Pearson
correlation coefficient lies between 0 and 1, the corresponding distance
lies between 0 and 1 as well.

In the context of gene expression experiments, the absolute correlation
is equal to 1 if the gene expression profiles of two genes are either
exactly the same or exactly opposite. The absolute correlation
coefficient should therefore be used with care.

### Uncentered correlation (cosine of the angle) {#uncentered-correlation-cosine-of-the-angle .unnumbered}

In some cases, it may be preferable to use the *uncentered correlation*
instead of the regular Pearson correlation coefficient. The uncentered
correlation is defined as
$$r_{\textrm U} = \frac{1}{n} \sum_{i=1}^{n} \left(\frac{x_i}{\sigma_x^{(0)}} \right) \left(\frac{y_i}{\sigma_y^{(0)}} \right),$$
where $$\begin{aligned}
\sigma_x^{(0)} & = & \sqrt{{\frac{1}{n}} \sum_{i=1}^{n}x_i^2}; \nonumber \\
\sigma_y^{(0)} & = & \sqrt{{\frac{1}{n}} \sum_{i=1}^{n}y_i^2}. \nonumber\end{aligned}$$
This is the same expression as for the regular Pearson correlation
coefficient, except that the sample means $\bar{x}, \bar{y}$ are set
equal to zero. The uncentered correlation may be appropriate if there is
a zero reference state. For instance, in the case of gene expression
data given in terms of log-ratios, a log-ratio equal to zero corresponds
to the green and red signal being equal, which means that the
experimental manipulation did not affect the gene expression.

The distance corresponding to the uncentered correlation coefficient is
defined as $$d_{\mbox{U}} \equiv 1 - r_{\mbox{U}},$$ where
$r_{\mbox{U}}$ is the uncentered correlation. As the uncentered
correlation coefficient lies between -1 and 1, the corresponding
distance lies between 0 and 2.

The uncentered correlation is equal to the cosine of the angle of the
two data vectors in $n$-dimensional space, and is often referred to as
such.

### Absolute uncentered correlation {#absolute-uncentered-correlation .unnumbered}

As for the regular Pearson correlation, we can define a distance measure
using the absolute value of the uncentered correlation:
$$d_{\mbox{AU}} \equiv 1 - \left|r_{\mbox{U}}\right|,$$ where
$r_{\mbox{U}}$ is the uncentered correlation coefficient. As the
absolute value of the uncentered correlation coefficient lies between 0
and 1, the corresponding distance lies between 0 and 1 as well.

Geometrically, the absolute value of the uncentered correlation is equal
to the cosine between the supporting lines of the two data vectors
(i.e., the angle without taking the direction of the vectors into
consideration).

### Spearman rank correlation {#spearman-rank-correlation .unnumbered}

The Spearman rank correlation is an example of a non-parametric
similarity measure, and tends to be more robust against outliers than
the Pearson correlation.

To calculate the Spearman rank correlation, we replace each data value
by their rank if we would order the data in each vector by their value.
We then calculate the Pearson correlation between the two rank vectors
instead of the data vectors.

As in the case of the Pearson correlation, we can define a distance
measure corresponding to the Spearman rank correlation as
$$d_{\mbox{S}} \equiv 1 - r_{\mbox{S}},$$ where $r_{\mbox{S}}$ is the
Spearman rank correlation.

### Kendall’s $\tau$ {#kendalls-tau .unnumbered}

Kendall’s $\tau$ is another example of a non-parametric similarity
measure. It is similar to the Spearman rank correlation, but instead of
the ranks themselves only the relative ranks are used to calculate
$\tau$ (see Snedecor & Cochran @snedecor1989).

We can define a distance measure corresponding to Kendall’s $\tau$ as
$$d_{\mbox{K}} \equiv 1 - \tau.$$ As Kendall’s $\tau$ is always between
-1 and 1, the corresponding distance will be between 0 and 2.

### Weighting {#weighting .unnumbered}

For most of the distance functions available in `Bio.Cluster`, a weight
vector can be applied. The weight vector contains weights for the items
in the data vector. If the weight for item $i$ is $w_i$, then that item
is treated as if it occurred $w_i$ times in the data. The weight do not
have to be integers. For the Spearman rank correlation and Kendall’s
$\tau$, weights do not have a well-defined meaning and are therefore not
implemented.

### Calculating the distance matrix {#subsec:distancematrix .unnumbered}

The distance matrix is a square matrix with all pairwise distances
between the items in `data`, and can be calculated by the function
`distancematrix` in the `Bio.Cluster` module:




Análise de cluster

 ================

 A análise de cluster é o agrupamento de itens em clusters com base no

 similaridade dos itens entre si. Em bioinformática, o agrupamento é

 amplamente utilizado na análise de dados de expressão gênica para encontrar grupos de genes

 com perfis de expressão gênica semelhantes. Isso pode identificar funcionalmente

 genes relacionados, bem como sugerem a função de genes atualmente desconhecidos

 genes.

 O módulo `Bio.Cluster` fornece agrupamento comumente usado

 algoritmos e foi projetado com a aplicação de dados de expressão gênica

 em mente. No entanto, este módulo também pode ser usado para análise de cluster de

 outros tipos de dados. `Bio.Cluster` e o cluster C subjacente

 A biblioteca é descrita por De Hoon *et al.* @ dehoon2004.

 As quatro abordagens de agrupamento a seguir são implementadas em

 `Bio.Cluster` :
-  Agrupamento hierárquico (centróide par a par, único, completo e ligação média);
-  $ k $ -means, $ k $ -medians e $ k $ -medians clustering;
-  Mapas auto-organizáveis;
-  Análise do componente principal.

###  Representação de dados {# data-representação .unnumbered}

 Os dados a serem agrupados são representados por um $ n \ times m $ Numérico

 `data` matriz Python. Dentro do contexto de dados de expressão gênica

 agrupamento, normalmente as linhas correspondem a genes diferentes, enquanto o

 colunas correspondem a diferentes condições experimentais. O agrupamento

 algoritmos em `Bio.Cluster` podem ser aplicados a linhas (genes) e

 colunas (experimentos).

###  Valores ausentes {# valores ausentes .unnumbered}

 Muitas vezes, em experimentos de microarray, alguns dos valores de dados estão ausentes,

 que é indicado por um adicional $ n \ times m $ Python Numérica

 `mask` matriz de inteiros. Se `mask[i,j]==0` , então os `data[i,j]` estão faltando e

 é ignorado na análise.

###  Gerador de número aleatório {# gerador de número aleatório. Não numerado}

 Os algoritmos de agrupamento $ k $ -means / medians / medoids e de auto-organização

 Mapas (SOMs) incluem o uso de um gerador de números aleatórios. O uniforme

 gerador de números aleatórios em `Bio.Cluster` é baseado no algoritmo de

 L&#39;Ecuyer @ lecuyer1988, enquanto os números aleatórios seguem o binômio

 distribuição são gerados usando o algoritmo BTPE por Kachitvichyanukul

 e Schmeiser @ kachitvichyanukul1988. O gerador de números aleatórios é

 inicializado automaticamente durante sua primeira chamada. Como este número aleatório

 gerador usa uma combinação de duas congruenciais lineares multiplicativas

 geradores, duas sementes (inteiras) são necessárias para a inicialização, para as quais

 usamos o gerador de número aleatório fornecido pelo sistema `rand` (no C

 biblioteca padrão). Inicializamos este gerador chamando `srand` com

 o tempo da época em segundos e use os dois primeiros números aleatórios

 gerado por `rand` como sementes para o gerador de número aleatório uniforme em

 `Bio.Cluster` .

 Funções de distância {#sec: distancefunctions}

 A fim de agrupar itens em grupos com base em sua similaridade, nós

 deve primeiro definir o que exatamente queremos dizer com *semelhante* . `Bio.Cluster`

 fornece oito funções de distância, indicadas por um único caractere, para

 medir semelhança ou, inversamente, distância:
-  `&#39;e&#39;` : distância euclidiana;
-  `&#39;b&#39;` : distância de quarteirão.
-  `&#39;c&#39;` : coeficiente de correlação de Pearson;
-  `&#39;a&#39;` : valor absoluto do coeficiente de correlação de Pearson;
-  `&#39;u&#39;` : Correlação de Pearson não centrada (equivalente ao cosseno de o ângulo entre dois vetores de dados);
-  `&#39;x&#39;` : Correlação absoluta de Pearson não centrada;
-  `&#39;s&#39;` : correlação de posto de Spearman;
-  `&#39;k&#39;` : $ \ tau $ de Kendall.

 Os dois primeiros são funções de distância verdadeiras que satisfazem o triângulo

 desigualdade:

 $$ d \ left (\ underline {u}, \ underline {v} \ right) \ leq d \ left (\ underline {u}, \ underline {w} \ right) + d \ left (\ underline {w} , \ underline {v} \ right) \ textrm {para todos} \ underline {u}, \ underline {v}, \ underline {w}, $$

 e, portanto, são chamados de *métricas* . Na linguagem cotidiana, este

 significa que a distância mais curta entre dois pontos é uma linha reta.

 As seis medidas de distância restantes estão relacionadas à correlação

 coeficiente, onde a distância $ d $ é definida em termos de

 correlação $ r $ por $ d = 1-r $. Observe que essas funções de distância são

 *semimétricas* que não satisfazem a desigualdade do triângulo. Por exemplo,

 para $$ \ underline {u} = \ left (1,0, -1 \ right); $$

 $$ \ underline {v} = \ left (1,1,0 \ right); $$

 $$ \ underline {w} = \ left (0,1,1 \ right); $$ encontramos uma distância de Pearson

 $ d \ left (\ underline {u}, \ underline {w} \ right) = 1,8660 $, enquanto

 $ d \ left (\ underline {u}, \ underline {v} \ right) + d \ left (\ underline {v}, \ underline {w} \ right) = 1,6340 $.

###  Distância euclidiana {# distância euclidiana. Não numerada}

 Em `Bio.Cluster` , definimos a distância euclidiana como

 $$ d = {1 \ over n} \ sum_ {i = 1} ^ {n} \ left (x_i-y_i \ right) ^ {2}. $$ Apenas aqueles

 termos estão incluídos na soma para a qual $ x_i $ e $ y_i $ são

 presente, e o denominador $ n $ é escolhido de acordo. Enquanto o

 dados de expressão $ x_i $ e $ y_i $ são subtraídos diretamente um do outro,

 devemos ter certeza de que os dados da expressão estão devidamente normalizados

 ao usar a distância euclidiana.

###  Distância do quarteirão da cidade {# distância do quarteirão da cidade .unnumbered}

 A distância do quarteirão, também conhecida como distância de Manhattan,

 está relacionado à distância euclidiana. Considerando que a distância euclidiana

 corresponde ao comprimento do caminho mais curto entre dois pontos, o

 a distância do quarteirão da cidade é a soma das distâncias ao longo de cada dimensão. Como

 dados de expressão gênica tendem a ter valores ausentes, em `Bio.Cluster` nós

 definir a distância do quarteirão como a soma das distâncias divididas pelo

 número de dimensões:

 $$ d = {1 \ over n} \ sum_ {i = 1} ^ n \ left | x_i-y_i \ right |. $$ Isso é igual a

 a distância que você teria que andar entre dois pontos em uma cidade, onde

 você tem que caminhar pelos quarteirões da cidade. Quanto à distância euclidiana, o

 os dados da expressão são subtraídos diretamente uns dos outros, e devemos

 portanto, certifique-se de que eles estão normalizados corretamente.

###  O coeficiente de correlação de Pearson {# coeficiente de correlação de Pearson. Não numerado}

 O coeficiente de correlação de Pearson é definido como

 $$ r = \ frac {1} {n} \ sum_ {i = 1} ^ n \ left (\ frac {x_i - \ bar {x}} {\ sigma_x} \ right) \ left (\ frac {y_i - \ bar {y}} {\ sigma_y} \ right), $$

 em que $ \ bar {x}, \ bar {y} $ são a média da amostra de $ x $ e $ y $

 respectivamente, e $ \ sigma_x, \ sigma_y $ são o desvio padrão da amostra

 de $ x $ e $ y $. O coeficiente de correlação de Pearson é uma medida de como

 bem, uma linha reta pode ser ajustada a um gráfico de dispersão de $ x $ e $ y $. E se

 todos os pontos no gráfico de dispersão estão em uma linha reta, o Pearson

 coeficiente de correlação é +1 ou -1, dependendo se o

 inclinação da linha é positiva ou negativa. Se a correlação de Pearson

 coeficiente é igual a zero, não há correlação entre $ x $ e

 $ y $.

 A *distância de Pearson* é então definida como

 $$ d _ {\ textrm {P}} \ equiv 1 - r. $$ Como o coeficiente de correlação de Pearson

 fica entre -1 e 1, a distância de Pearson fica entre 0 e 2.

###  Correlação absoluta de Pearson {# absolute-pearson-correlation .unnumbered}

 Ao tomar o valor absoluto da correlação de Pearson, encontramos um

 número entre 0 e 1. Se o valor absoluto for 1, todos os pontos em

 o gráfico de dispersão está em uma linha reta com um positivo ou um

 inclinação negativa. Se o valor absoluto for igual a zero, não há

 correlação entre $ x $ e $ y $.

 A distância correspondente é definida como

 $$ d _ {\ textrm A} \ equiv 1 - \ left | r \ right |, $$ onde $ r $ é o Pearson

 coeficiente de correlação. Como o valor absoluto do Pearson

 coeficiente de correlação está entre 0 e 1, a distância correspondente

 também está entre 0 e 1.

 No contexto de experimentos de expressão gênica, a correlação absoluta

 é igual a 1 se os perfis de expressão gênica de dois genes forem

 exatamente o mesmo ou exatamente oposto. A correlação absoluta

 coeficiente deve, portanto, ser usado com cuidado.

###  Correlação não centrada (cosseno do ângulo) {# correlação-cosseno-do-ângulo não centrada. Não numerada}

 Em alguns casos, pode ser preferível usar a *correlação não centrada*

 em vez do coeficiente de correlação regular de Pearson. O descentrado

 correlação é definida como

 $$ r *{\ textrm U} = \ frac {1} {n} \ sum* {i = 1} ^ {n} \ left (\ frac {x_i} {\ sigma_x ^ {(0)}} \ right) \ esquerda (\ frac {y_i} {\ sigma_y ^ {(0)}} \ direita), $$

 onde $$ \ begin {alinhado}

 \ sigma *x ^ {(0)} &amp; = &amp; \ sqrt {{\ frac {1} {n}} \ sum* {i = 1} ^ {n} x_i ^ 2}; \nenhum número \

 \ sigma *y ^ {(0)} &amp; = &amp; \ sqrt {{\ frac {1} {n}} \ sum* {i = 1} ^ {n} y_i ^ 2}. \ nonumber \ end {alinhado} $$

 Esta é a mesma expressão que para a correlação regular de Pearson

 coeficiente, exceto que a amostra significa que $ \ bar {x}, \ bar {y} $ estão definidos

 igual a zero. A correlação não centrada pode ser apropriada se houver

 um estado de referência zero. Por exemplo, no caso da expressão gênica

 dados dados em termos de relações logarítmicas, uma razão logarítmica igual a zero corresponde

 ao sinal verde e vermelho sendo iguais, o que significa que o

 a manipulação experimental não afetou a expressão gênica.

 A distância correspondente ao coeficiente de correlação não centrado é

 definido como $$ d *{\ mbox {U}} \ equiv 1 - r* {\ mbox {U}}, $$ onde

 $ r _ {\ mbox {U}} $ é a correlação não centrada. Como o descentrado

 coeficiente de correlação está entre -1 e 1, o correspondente

 a distância está entre 0 e 2.

 A correlação não centrada é igual ao cosseno do ângulo do

 dois vetores de dados no espaço $ n $ -dimensional, e é frequentemente referido como

 tal.

###  Correlação absoluta não centrada {# correlação absoluta não centrada. Não numerada}

 Quanto à correlação regular de Pearson, podemos definir uma medida de distância

 usando o valor absoluto da correlação não centrada:

 $$ d *{\ mbox {AU}} \ equiv 1 - \ left | r* {\ mbox {U}} \ right |, $$ onde

 $ r _ {\ mbox {U}} $ é o coeficiente de correlação não centrado. Enquanto o

 o valor absoluto do coeficiente de correlação não centrado está entre 0

 e 1, a distância correspondente também está entre 0 e 1.

 Geometricamente, o valor absoluto da correlação não centrada é igual

 ao cosseno entre as linhas de suporte dos dois vetores de dados

 (ou seja, o ângulo sem tomar a direção dos vetores em

 consideração).

###  Correlação de classificação de Spearman {# correlação de classificação de spearman .unnumbered}

 A correlação de classificação de Spearman é um exemplo de um método não paramétrico

 medida de similaridade, e tende a ser mais robusta contra outliers do que

 a correlação de Pearson.

 Para calcular a correlação de classificação de Spearman, substituímos cada valor de dados

 por sua classificação, se ordenássemos os dados em cada vetor por seu valor.

 Em seguida, calculamos a correlação de Pearson entre os dois vetores de classificação

 em vez dos vetores de dados.

 Como no caso da correlação de Pearson, podemos definir uma distância

 medida correspondente à correlação de classificação de Spearman como

 $$ d *{\ mbox {S}} \ equiv 1 - r* {\ mbox {S}}, $$ onde $ r _ {\ mbox {S}} $ é o

 Correlação de classificação de Spearman.

###  $ \ Tau $ {# kendalls-tau .unnumbered} de Kendall

 $ \ Tau $ de Kendall é outro exemplo de similaridade não paramétrica

 a medida. É semelhante à correlação de classificação de Spearman, mas em vez de

 as próprias classificações, apenas as classificações relativas são usadas para calcular

 $ \ tau $ (consulte Snedecor &amp; Cochran @ snedecor1989).

 Podemos definir uma medida de distância correspondente a $ \ tau $ de Kendall como

 $$ d _ {\ mbox {K}} \ equiv 1 - \ tau. $$ Como $ \ tau $ de Kendall está sempre entre

 -1 e 1, a distância correspondente será entre 0 e 2.

###  Ponderação {#weighting .unnumbered}

 Para a maioria das funções de distância disponíveis no `Bio.Cluster` , um peso

 o vetor pode ser aplicado. O vetor de peso contém pesos para os itens

 no vetor de dados. Se o peso do item $ i $ for $ w_i $, então esse item

 é tratado como se tivesse ocorrido $ w_i $ vezes nos dados. O peso não

 têm que ser inteiros. Para a correlação de classificação de Spearman e Kendall&#39;s

 $ \ tau $, os pesos não têm um significado bem definido e, portanto, não são

 implementado.

###  Calculando a matriz de distâncias {#subsec: distancematrix .unnumbered}

 A matriz de distância é uma matriz quadrada com todas as distâncias de pares

 entre os itens de `data` e podem ser calculados pela função

 `distancematrix` no módulo `Bio.Cluster` :


In [1]:
from Bio.Cluster import distancematrix
matrix = distancematrix(data)

NameError: name 'data' is not defined


where the following arguments are defined:

-   `data` (required)\
    Array containing the data for the items.

-   `mask` (default: `None`)\
    Array of integers showing which data are missing. If `mask[i,j]==0`,
    then `data[i,j]` is missing. If `mask==None`, then all data
    are present.

-   `weight` (default: `None`)\
    The weights to be used when calculating distances. If
    `weight==None`, then equal weights are assumed.

-   `transpose` (default: `0`)\
    Determines if the distances between the rows of `data` are to be
    calculated (`transpose==0`), or between the columns of `data`
    (`transpose==1`).

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]).

To save memory, the distance matrix is returned as a list of 1D arrays.
The number of columns in each row is equal to the row number. Hence, the
first row has zero elements. An example of the return value is




onde os seguintes argumentos são definidos:
-  `data` (obrigatório) \ Matriz contendo os dados dos itens.
-  `mask` (padrão: `None` ) \ Matriz de inteiros mostrando quais dados estão faltando. Se `mask[i,j]==0` , então os `data[i,j]` estão faltando. Se `mask==None` , então todos os dados estão presentes.
-  `weight` (padrão: `None` ) \ Os pesos a serem usados ao calcular distâncias. E se `weight==None` , então pesos iguais são assumidos.
-  `transpose` (padrão: `0` ) \ Determina se as distâncias entre as linhas de `data` devem ser calculado ( `transpose==0` ), ou entre as colunas de `data` ( `transpose==1` ).
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]).

 Para economizar memória, a matriz de distância é retornada como uma lista de matrizes 1D.

 O número de colunas em cada linha é igual ao número da linha. Portanto, o

 a primeira linha tem zero elementos. Um exemplo do valor de retorno é


```
[array([]),
 array([1.]),
 array([7., 3.]),
 array([4., 2., 6.])]

```


```
[array([]), array([1.]), array([7., 3.]), array([4., 2., 6.])]
```



This corresponds to the distance matrix $$\left(
\begin{array}{cccc}
0 & 1 & 7 & 4  \\
1 & 0 & 3 & 2  \\
7 & 3 & 0 & 6  \\
4 & 2 & 6 & 0
\end{array}
\right).$$

Calculating cluster properties
------------------------------

### Calculating the cluster centroids {#subsec:clustercentroids .unnumbered}

The centroid of a cluster can be defined either as the mean or as the
median of each dimension over all cluster items. The function
`clustercentroids` in `Bio.Cluster` can be used to calculate either:




Isso corresponde à matriz de distância $$ \ left (

 \ begin {array} {cccc}

 0 e 1 e 7 e 4 \

 1 e 0 e 3 e 2 \

 7 e 3 e 0 e 6 \

 4 e 2 e 6 e 0

 \ end {array}

 \ certo). $$

 Calculando propriedades do cluster

###  Calculando os centróides do cluster {#subsec: clustercentroids .unnumbered}

 O centróide de um cluster pode ser definido como a média ou como o

 mediana de cada dimensão sobre todos os itens do cluster. A função

 `clustercentroids` em `Bio.Cluster` podem ser usados para calcular:


In [2]:
from Bio.Cluster import clustercentroids
cdata, cmask = clustercentroids(data)

NameError: name 'data' is not defined


where the following arguments are defined:

-   `data` (required)\
    Array containing the data for the items.

-   `mask` (default: `None`)\
    Array of integers showing which data are missing. If `mask[i,j]==0`,
    then `data[i,j]` is missing. If `mask==None`, then all data
    are present.

-   `clusterid` (default: `None`)\
    Vector of integers showing to which cluster each item belongs. If
    `clusterid` is `None`, then all items are assumed to belong to the
    same cluster.

-   `method` (default: `'a'`)\
    Specifies whether the arithmetic mean (`method=='a'`) or the median
    (`method=='m'`) is used to calculate the cluster center.

-   `transpose` (default: `0`)\
    Determines if the centroids of the rows of `data` are to be
    calculated (`transpose==0`), or the centroids of the columns of
    `data` (`transpose==1`).

This function returns the tuple `(cdata, cmask)`. The centroid data are
stored in the 2D Numerical Python array `cdata`, with missing data
indicated by the 2D Numerical Python integer array `cmask`. The
dimensions of these arrays are
$\left(\textrm{number of clusters}, \textrm{number of columns}\right)$
if `transpose` is `0`, or
$\left(\textrm{number of rows}, \textrm{number of clusters}\right)$ if
`transpose` is `1`. Each row (if `transpose` is `0`) or column (if
`transpose` is `1`) contains the averaged data corresponding to the
centroid of each cluster.

### Calculating the distance between clusters {#calculating-the-distance-between-clusters .unnumbered}

Given a distance function between *items*, we can define the distance
between two *clusters* in several ways. The distance between the
arithmetic means of the two clusters is used in pairwise
centroid-linkage clustering and in $k$-means clustering. In $k$-medoids
clustering, the distance between the medians of the two clusters is used
instead. The shortest pairwise distance between items of the two
clusters is used in pairwise single-linkage clustering, while the
longest pairwise distance is used in pairwise maximum-linkage
clustering. In pairwise average-linkage clustering, the distance between
two clusters is defined as the average over the pairwise distances.

To calculate the distance between two clusters, use




onde os seguintes argumentos são definidos:
-  `data` (obrigatório) \ Matriz contendo os dados dos itens.
-  `mask` (padrão: `None` ) \ Matriz de inteiros mostrando quais dados estão faltando. Se `mask[i,j]==0` , então os `data[i,j]` estão faltando. Se `mask==None` , então todos os dados estão presentes.
-  `clusterid` (padrão: `None` ) \ Vetor de inteiros mostrando a qual cluster cada item pertence. E se `clusterid` é `None` , então todos os itens são considerados pertencentes ao mesmo cluster.
-  `method` (padrão: `&#39;a&#39;` ) \ Especifica se a média aritmética ( `method==&#39;a&#39;` ) ou a mediana ( `method==&#39;m&#39;` ) é usado para calcular o centro do cluster.
-  `transpose` (padrão: `0` ) \ Determina se os centróides das linhas de `data` devem ser calculado ( `transpose==0` ), ou os centróides das colunas de `data` ( `transpose==1` ).

 Esta função retorna a tupla `(cdata, cmask)` . Os dados do centróide são

 armazenados no `cdata` matriz Python Numérica 2D, com dados ausentes

 indicado pelo `cmask` array inteiro 2D numérico Python. o

 dimensões dessas matrizes são

 $ \ left (\ textrm {número de clusters}, \ textrm {número de colunas} \ right) $

 se `transpose` for `0` , ou

 $ \ left (\ textrm {número de linhas}, \ textrm {número de clusters} \ right) $ se

 `transpose` é `1` . Cada linha (se a `transpose` for `0` ) ou coluna (se

 `transpose` é `1` ) contém os dados médios correspondentes ao

 centróide de cada cluster.

###  Calculando a distância entre clusters {# calculating-the-distance-between-clusters .unnumbered}

 Dada uma função de distância entre os *itens* , podemos definir a distância

 entre dois *clusters* de várias maneiras. A distância entre o

 a média aritmética dos dois clusters é usada em pares

 agrupamento de ligação centróide e em $ k $ -means agrupamento. Em $ k $ -medóides

 clustering, a distância entre as medianas dos dois clusters é usada

 em vez de. A menor distância entre pares entre os itens dos dois

 clusters é usado em clustering de ligação única par, enquanto o

 a distância de pares mais longa é usada na ligação máxima de pares

 agrupamento. No agrupamento de ligação média pareada, a distância entre

 dois clusters são definidos como a média das distâncias entre pares.

 Para calcular a distância entre dois clusters, use


In [3]:
from Bio.Cluster import clusterdistance
distance = clusterdistance(data)

NameError: name 'data' is not defined


where the following arguments are defined:

-   `data` (required)\
    Array containing the data for the items.

-   `mask` (default: `None`)\
    Array of integers showing which data are missing. If `mask[i,j]==0`,
    then `data[i,j]` is missing. If `mask==None`, then all data
    are present.

-   `weight` (default: `None`)\
    The weights to be used when calculating distances. If
    `weight==None`, then equal weights are assumed.

-   `index1` (default: `0`)\
    A list containing the indices of the items belonging to the
    first cluster. A cluster containing only one item $i$ can be
    represented either as a list `[i]`, or as an integer `i`.

-   `index2` (default: `0`)\
    A list containing the indices of the items belonging to the
    second cluster. A cluster containing only one items $i$ can be
    represented either as a list `[i]`, or as an integer `i`.

-   `method` (default: `'a'`)\
    Specifies how the distance between clusters is defined:

    -   `'a'`: Distance between the two cluster centroids (arithmetic
        mean);

    -   `'m'`: Distance between the two cluster centroids (median);

    -   `'s'`: Shortest pairwise distance between items in the two
        clusters;

    -   `'x'`: Longest pairwise distance between items in the two
        clusters;

    -   `'v'`: Average over the pairwise distances between items in the
        two clusters.

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]).

-   `transpose` (default: `0`)\
    If `transpose==0`, calculate the distance between the rows of
    `data`. If `transpose==1`, calculate the distance between the
    columns of `data`.

Partitioning algorithms
-----------------------

Partitioning algorithms divide items into $k$ clusters such that the sum
of distances over the items to their cluster centers is minimal. The
number of clusters $k$ is specified by the user. Three partitioning
algorithms are available in `Bio.Cluster`:

-   $k$-means clustering

-   $k$-medians clustering

-   $k$-medoids clustering

These algorithms differ in how the cluster center is defined. In
$k$-means clustering, the cluster center is defined as the mean data
vector averaged over all items in the cluster. Instead of the mean, in
$k$-medians clustering the median is calculated for each dimension in
the data vector. Finally, in $k$-medoids clustering the cluster center
is defined as the item which has the smallest sum of distances to the
other items in the cluster. This clustering algorithm is suitable for
cases in which the distance matrix is known but the original data matrix
is not available, for example when clustering proteins based on their
structural similarity.

The expectation-maximization (EM) algorithm is used to find this
partitioning into $k$ groups. In the initialization of the EM algorithm,
we randomly assign items to clusters. To ensure that no empty clusters
are produced, we use the binomial distribution to randomly choose the
number of items in each cluster to be one or more. We then randomly
permute the cluster assignments to items such that each item has an
equal probability to be in any cluster. Each cluster is thus guaranteed
to contain at least one item.

We then iterate:

-   Calculate the centroid of each cluster, defined as either the mean,
    the median, or the medoid of the cluster;

-   Calculate the distances of each item to the cluster centers;

-   For each item, determine which cluster centroid is closest;

-   Reassign each item to its closest cluster, or stop the iteration if
    no further item reassignments take place.

To avoid clusters becoming empty during the iteration, in $k$-means and
$k$-medians clustering the algorithm keeps track of the number of items
in each cluster, and prohibits the last remaining item in a cluster from
being reassigned to a different cluster. For $k$-medoids clustering,
such a check is not needed, as the item that functions as the cluster
centroid has a zero distance to itself, and will therefore never be
closer to a different cluster.

As the initial assignment of items to clusters is done randomly, usually
a different clustering solution is found each time the EM algorithm is
executed. To find the optimal clustering solution, the $k$-means
algorithm is repeated many times, each time starting from a different
initial random clustering. The sum of distances of the items to their
cluster center is saved for each run, and the solution with the smallest
value of this sum will be returned as the overall clustering solution.

How often the EM algorithm should be run depends on the number of items
being clustered. As a rule of thumb, we can consider how often the
optimal solution was found; this number is returned by the partitioning
algorithms as implemented in this library. If the optimal solution was
found many times, it is unlikely that better solutions exist than the
one that was found. However, if the optimal solution was found only
once, there may well be other solutions with a smaller within-cluster
sum of distances. If the number of items is large (more than several
hundreds), it may be difficult to find the globally optimal solution.

The EM algorithm terminates when no further reassignments take place. We
noticed that for some sets of initial cluster assignments, the EM
algorithm fails to converge due to the same clustering solution
reappearing periodically after a small number of iteration steps. We
therefore check for the occurrence of such periodic solutions during the
iteration. After a given number of iteration steps, the current
clustering result is saved as a reference. By comparing the clustering
result after each subsequent iteration step to the reference state, we
can determine if a previously encountered clustering result is found. In
such a case, the iteration is halted. If after a given number of
iterations the reference state has not yet been encountered, the current
clustering solution is saved to be used as the new reference state.
Initially, ten iteration steps are executed before resaving the
reference state. This number of iteration steps is doubled each time, to
ensure that periodic behavior with longer periods can also be detected.

### $k$-means and $k$-medians {#k-means-and-k-medians .unnumbered}

The $k$-means and $k$-medians algorithms are implemented as the function
`kcluster` in `Bio.Cluster`:




onde os seguintes argumentos são definidos:
-  `data` (obrigatório) \ Matriz contendo os dados dos itens.
-  `mask` (padrão: `None` ) \ Matriz de inteiros mostrando quais dados estão faltando. Se `mask[i,j]==0` , então os `data[i,j]` estão faltando. Se `mask==None` , então todos os dados estão presentes.
-  `weight` (padrão: `None` ) \ Os pesos a serem usados ao calcular distâncias. E se `weight==None` , então pesos iguais são assumidos.
-  `index1` (padrão: `0` ) \ Uma lista contendo os índices dos itens pertencentes ao primeiro cluster. Um cluster contendo apenas um item $ i $ pode ser representado como uma lista `[i]` ou como um inteiro `i` .
-  `index2` (padrão: `0` ) \ Uma lista contendo os índices dos itens pertencentes ao segundo cluster. Um cluster contendo apenas um item $ i $ pode ser representado como uma lista `[i]` ou como um inteiro `i` .
-  `method` (padrão: `&#39;a&#39;` ) \ Especifica como a distância entre os clusters é definida:
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]).
-  `transpose` (padrão: `0` ) \ Se `transpose==0` , calcule a distância entre as linhas de `data` . Se `transpose==1` , calcule a distância entre os colunas de `data` .

 Algoritmos de particionamento

 Os algoritmos de particionamento dividem os itens em $ k $ clusters de forma que a soma

 de distâncias sobre os itens para seus centros de cluster é mínimo. o

 número de clusters $ k $ é especificado pelo usuário. Três partições

 algoritmos estão disponíveis no `Bio.Cluster` :
-  $ k $ - significa agrupamento
-  Agrupamento de $ k $ -medians
-  clustering de $ k $ -medoids

 Esses algoritmos diferem em como o centro do cluster é definido. Dentro

 $ k $ -means clustering, o centro do cluster é definido como os dados médios

 a média do vetor de todos os itens do cluster. Em vez da média, em

 $ k $ -medians agrupando a mediana é calculado para cada dimensão em

 o vetor de dados. Finalmente, em $ k $ -medoids agrupando o centro do cluster

 é definido como o item que tem a menor soma das distâncias ao

 outros itens do cluster. Este algoritmo de agrupamento é adequado para

 casos em que a matriz de distância é conhecida, mas a matriz de dados original

 não está disponível, por exemplo, ao agrupar proteínas com base em seus

 similaridade estrutural.

 O algoritmo de maximização de expectativa (EM) é usado para encontrar este

 particionamento em grupos $ k $. Na inicialização do algoritmo EM,

 atribuímos itens aleatoriamente aos clusters. Para garantir que nenhum cluster vazio

 são produzidos, usamos a distribuição binomial para escolher aleatoriamente o

 número de itens em cada cluster seja um ou mais. Nós então aleatoriamente

 permutar as atribuições de cluster para itens de modo que cada item tenha um

 probabilidade igual de estar em qualquer cluster. Cada cluster é, portanto, garantido

 para conter pelo menos um item.

 Em seguida, iteramos:
-  Calcule o centroide de cada cluster, definido como a média, o mediano, ou o medóide do cluster;
-  Calcule as distâncias de cada item aos centros do cluster;
-  Para cada item, determine qual centróide do cluster está mais próximo;
-  Reatribua cada item ao seu cluster mais próximo ou pare a iteração se nenhuma outra reatribuição de itens ocorre.

 Para evitar que os clusters fiquem vazios durante a iteração, em $ k $ - significa e

 $ k $ -medians agrupando o algoritmo mantém o controle do número de itens

 em cada cluster, e proíbe o último item restante em um cluster de

 sendo reatribuído a um cluster diferente. Para agrupamento de $ k $ -medoids,

 tal verificação não é necessária, pois o item que funciona como o cluster

 o centróide tem uma distância zero de si mesmo e, portanto, nunca será

 mais perto de um cluster diferente.

 Como a atribuição inicial de itens aos clusters é feita aleatoriamente, geralmente

 uma solução de cluster diferente é encontrada cada vez que o algoritmo EM é

 executado. Para encontrar a solução de agrupamento ideal, o $ k $ significa

 algoritmo é repetido muitas vezes, cada vez começando de um

 agrupamento inicial aleatório. A soma das distâncias dos itens ao seu

 o centro do cluster é salvo para cada execução, e a solução com o menor

 valor dessa soma será retornado como a solução geral de clustering.

 A frequência com que o algoritmo EM deve ser executado depende do número de itens

 sendo agrupados. Como regra geral, podemos considerar a frequência com que o

 solução ótima foi encontrada; este número é retornado pelo particionamento

 algoritmos conforme implementados nesta biblioteca. Se a solução ideal fosse

 encontrado muitas vezes, é improvável que existam soluções melhores do que o

 aquele que foi encontrado. No entanto, se a solução ideal foi encontrada apenas

 uma vez, pode muito bem haver outras soluções com um menor dentro do cluster

 soma das distâncias. Se o número de itens for grande (mais de vários

 centenas), pode ser difícil encontrar a solução globalmente ideal.

 O algoritmo EM termina quando não ocorrem mais reatribuições. Nós

 notou que para alguns conjuntos de atribuições de cluster iniciais, o EM

 algoritmo falha em convergir devido à mesma solução de clustering

 reaparecer periodicamente após um pequeno número de etapas de iteração. Nós

 portanto, verifique a ocorrência de tais soluções periódicas durante o

 iteração. Após um determinado número de etapas de iteração, o atual

 o resultado do agrupamento é salvo como uma referência. Comparando o agrupamento

 resultado após cada etapa de iteração subsequente para o estado de referência, nós

 pode determinar se um resultado de cluster encontrado anteriormente foi encontrado. Dentro

 nesse caso, a iteração é interrompida. Se após um determinado número de

 iterações o estado de referência ainda não foi encontrado, o atual

 solução de cluster é salva para ser usada como o novo estado de referência.

 Inicialmente, dez etapas de iteração são executadas antes de salvar novamente o

 estado de referência. Este número de etapas de iteração é dobrado a cada vez, para

 garantir que comportamentos periódicos com períodos mais longos também possam ser detectados.

###  $ k $ -médios e $ k $ -medianos {# k-médias-e-k-medianas. não numerado}

 Os algoritmos $ k $ -means e $ k $ -medians são implementados como a função

 `kcluster` em `Bio.Cluster` :


In [4]:
from Bio.Cluster import kcluster
clusterid, error, nfound = kcluster(data)

NameError: name 'data' is not defined


where the following arguments are defined:

-   `data` (required)\
    Array containing the data for the items.

-   `nclusters` (default: `2`)\
    The number of clusters $k$.

-   `mask` (default: `None`)\
    Array of integers showing which data are missing. If `mask[i,j]==0`,
    then `data[i,j]` is missing. If `mask==None`, then all data
    are present.

-   `weight` (default: `None`)\
    The weights to be used when calculating distances. If
    `weight==None`, then equal weights are assumed.

-   `transpose` (default: `0`)\
    Determines if rows (`transpose` is `0`) or columns (`transpose` is
    `1`) are to be clustered.

-   `npass` (default: `1`)\
    The number of times the $k$-means/-medians clustering algorithm is
    performed, each time with a different (random) initial condition. If
    `initialid` is given, the value of `npass` is ignored and the
    clustering algorithm is run only once, as it behaves
    deterministically in that case.

-   `method` (default: `a`)\
    describes how the center of a cluster is found:

    -   `method=='a'`: arithmetic mean ($k$-means clustering);

    -   `method=='m'`: median ($k$-medians clustering).

    For other values of `method`, the arithmetic mean is used.

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]). Whereas all eight distance measures
    are accepted by `kcluster`, from a theoretical viewpoint it is best
    to use the Euclidean distance for the $k$-means algorithm, and the
    city-block distance for $k$-medians.

-   `initialid` (default: `None`)\
    Specifies the initial clustering to be used for the EM algorithm. If
    `initialid==None`, then a different random initial clustering is
    used for each of the `npass` runs of the EM algorithm. If
    `initialid` is not `None`, then it should be equal to a 1D array
    containing the cluster number (between `0` and `nclusters-1`) for
    each item. Each cluster should contain at least one item. With the
    initial clustering specified, the EM algorithm is deterministic.

This function returns a tuple `(clusterid, error, nfound)`, where
`clusterid` is an integer array containing the number of the cluster to
which each row or cluster was assigned, `error` is the within-cluster
sum of distances for the optimal clustering solution, and `nfound` is
the number of times this optimal solution was found.

### $k$-medoids clustering {#k-medoids-clustering .unnumbered}

The `kmedoids` routine performs $k$-medoids clustering on a given set of
items, using the distance matrix and the number of clusters passed by
the user:




onde os seguintes argumentos são definidos:
-  `data` (obrigatório) \ Matriz contendo os dados dos itens.
-  `nclusters` (padrão: `2` ) \ O número de clusters $ k $.
-  `mask` (padrão: `None` ) \ Matriz de inteiros mostrando quais dados estão faltando. Se `mask[i,j]==0` , então os `data[i,j]` estão faltando. Se `mask==None` , então todos os dados estão presentes.
-  `weight` (padrão: `None` ) \ Os pesos a serem usados ao calcular distâncias. E se `weight==None` , então pesos iguais são assumidos.
-  `transpose` (padrão: `0` ) \ Determina se as linhas ( `transpose` é `0` ) ou colunas ( `transpose` é `1` ) devem ser agrupados.
-  `npass` (padrão: `1` ) \ O número de vezes que o algoritmo de agrupamento $ k $ -means / -medians é realizada, cada vez com uma condição inicial diferente (aleatória). E se `initialid` é fornecido, o valor de `npass` é ignorado e o algoritmo de agrupamento é executado apenas uma vez, como se comporta deterministicamente nesse caso.
-  `method` (padrão: `a` ) \ descreve como o centro de um cluster é encontrado:
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]). Considerando que todas as oito medidas de distância são aceitos pelo `kcluster` , do ponto de vista teórico é melhor para usar a distância euclidiana para o algoritmo $ k $ -means, e o distância de quarteirão por $ k $ -medianos.
-  `initialid` (padrão: `None` ) \ Especifica o clustering inicial a ser usado para o algoritmo EM. E se `initialid==None` , então um agrupamento inicial aleatório diferente é usado para cada uma das execuções `npass` do algoritmo EM. E se `initialid` não é `None` , então deve ser igual a uma matriz 1D contendo o número do cluster (entre `0` e `nclusters-1` ) para cada item. Cada cluster deve conter pelo menos um item. Com o clustering inicial especificado, o algoritmo EM é determinístico.

 Esta função retorna uma tupla `(clusterid, error, nfound)` , onde

 `clusterid` é uma matriz inteira contendo o número do cluster para

 a qual cada linha ou cluster foi atribuído, o `error` é o dentro do cluster

 soma das distâncias para a solução de cluster ideal, e `nfound` é

 o número de vezes que essa solução ótima foi encontrada.

###  $ k $ -medoids clustering {# k-medoids-clustering .unnumbered}

 A rotina `kmedoids` executa $ k $ -medoids agrupamento em um determinado conjunto de

 itens, usando a matriz de distância e o número de clusters que passaram por

 o usuário:


In [5]:
from Bio.Cluster import kmedoids
clusterid, error, nfound = kmedoids(distance)

NameError: name 'distance' is not defined


where the following arguments are defined: , nclusters=2, npass=1,
initialid=None)|

-   <span>`distance`</span> (required)\
    The matrix containing the distances between the items; this matrix
    can be specified in three ways:

    -   as a 2D Numerical Python array (in which only the left-lower
        part of the array will be accessed):




onde os seguintes argumentos são definidos:, nclusters = 2, npass = 1,

 initialid = Nenhum) |
-  &lt;span&gt; `distance` &lt;/span&gt; (obrigatório) \ A matriz contendo as distâncias entre os itens; esta matriz pode ser especificado de três maneiras:


```
distance = array([[0.0, 1.1, 2.3],
                          [1.1, 0.0, 4.5],
                          [2.3, 4.5, 0.0]])

```


```
distance = array([[0.0, 1.1, 2.3], [1.1, 0.0, 4.5], [2.3, 4.5, 0.0]])
```



    -   as a 1D Numerical Python array containing consecutively the
        distances in the left-lower part of the distance matrix:




```
- as a 1D Numerical Python array containing consecutively the distances in the left-lower part of the distance matrix:
```


```
distance = array([1.1, 2.3, 4.5])

```


```

distance = array([1.1, 2.3, 4.5])

```



    -   as a list containing the rows of the left-lower part of the
        distance matrix:




```
- as a list containing the rows of the left-lower part of the distance matrix:
```


```
distance = [array([]|,
                    array([1.1]),
                    array([2.3, 4.5])
                   ]

```


```
distance = [array([]|, array([1.1]), array([2.3, 4.5]) ]
```



    These three expressions correspond to the same distance matrix.

-   `nclusters` (default: `2`)\
    The number of clusters $k$.

-   `npass` (default: `1`)\
    The number of times the $k$-medoids clustering algorithm is
    performed, each time with a different (random) initial condition. If
    `initialid` is given, the value of `npass` is ignored, as the
    clustering algorithm behaves deterministically in that case.

-   `initialid` (default: `None`)\
    Specifies the initial clustering to be used for the EM algorithm. If
    `initialid==None`, then a different random initial clustering is
    used for each of the `npass` runs of the EM algorithm. If
    `initialid` is not `None`, then it should be equal to a 1D array
    containing the cluster number (between `0` and `nclusters-1`) for
    each item. Each cluster should contain at least one item. With the
    initial clustering specified, the EM algorithm is deterministic.

This function returns a tuple `(clusterid, error, nfound)`, where
`clusterid` is an array containing the number of the cluster to which
each item was assigned, `error` is the within-cluster sum of distances
for the optimal $k$-medoids clustering solution, and `nfound` is the
number of times the optimal solution was found. Note that the cluster
number in `clusterid` is defined as the item number of the item
representing the cluster centroid.

Hierarchical clustering
-----------------------

Hierarchical clustering methods are inherently different from the
$k$-means clustering method. In hierarchical clustering, the similarity
in the expression profile between genes or experimental conditions are
represented in the form of a tree structure. This tree structure can be
shown graphically by programs such as Treeview and Java Treeview, which
has contributed to the popularity of hierarchical clustering in the
analysis of gene expression data.

The first step in hierarchical clustering is to calculate the distance
matrix, specifying all the distances between the items to be clustered.
Next, we create a node by joining the two closest items. Subsequent
nodes are created by pairwise joining of items or nodes based on the
distance between them, until all items belong to the same node. A tree
structure can then be created by retracing which items and nodes were
merged. Unlike the EM algorithm, which is used in $k$-means clustering,
the complete process of hierarchical clustering is deterministic.

Several flavors of hierarchical clustering exist, which differ in how
the distance between subnodes is defined in terms of their members. In
`Bio.Cluster`, pairwise single, maximum, average, and centroid linkage
are available.

-   In pairwise single-linkage clustering, the distance between two
    nodes is defined as the shortest distance among the pairwise
    distances between the members of the two nodes.

-   In pairwise maximum-linkage clustering, alternatively known as
    pairwise complete-linkage clustering, the distance between two nodes
    is defined as the longest distance among the pairwise distances
    between the members of the two nodes.

-   In pairwise average-linkage clustering, the distance between two
    nodes is defined as the average over all pairwise distances between
    the items of the two nodes.

-   In pairwise centroid-linkage clustering, the distance between two
    nodes is defined as the distance between their centroids. The
    centroids are calculated by taking the mean over all the items in
    a cluster. As the distance from each newly formed node to existing
    nodes and items need to be calculated at each step, the computing
    time of pairwise centroid-linkage clustering may be significantly
    longer than for the other hierarchical clustering methods. Another
    peculiarity is that (for a distance measure based on the Pearson
    correlation), the distances do not necessarily increase when going
    up in the clustering tree, and may even decrease. This is caused by
    an inconsistency between the centroid calculation and the distance
    calculation when using the Pearson correlation: Whereas the Pearson
    correlation effectively normalizes the data for the distance
    calculation, no such normalization occurs for the
    centroid calculation.

For pairwise single-, complete-, and average-linkage clustering, the
distance between two nodes can be found directly from the distances
between the individual items. Therefore, the clustering algorithm does
not need access to the original gene expression data, once the distance
matrix is known. For pairwise centroid-linkage clustering, however, the
centroids of newly formed subnodes can only be calculated from the
original data and not from the distance matrix.

The implementation of pairwise single-linkage hierarchical clustering is
based on the SLINK algorithm (R. Sibson, 1973), which is much faster and
more memory-efficient than a straightforward implementation of pairwise
single-linkage clustering. The clustering result produced by this
algorithm is identical to the clustering solution found by the
conventional single-linkage algorithm. The single-linkage hierarchical
clustering algorithm implemented in this library can be used to cluster
large gene expression data sets, for which conventional hierarchical
clustering algorithms fail due to excessive memory requirements and
running time.

### Representing a hierarchical clustering solution {#representing-a-hierarchical-clustering-solution .unnumbered}

The result of hierarchical clustering consists of a tree of nodes, in
which each node joins two items or subnodes. Usually, we are not only
interested in which items or subnodes are joined at each node, but also
in their similarity (or distance) as they are joined. To store one node
in the hierarchical clustering tree, we make use of the class `Node`,
which defined in `Bio.Cluster`. An instance of `Node` has three
attributes:

-   `left`

-   `right`

-   `distance`

Here, `left` and `right` are integers referring to the two items or
subnodes that are joined at this node, and `distance` is the distance
between them. The items being clustered are numbered from 0 to
$\left(\textrm{number of items} - 1\right)$, while clusters are numbered
from -1 to $-\left(\textrm{number of items}-1\right)$. Note that the
number of nodes is one less than the number of items.

To create a new `Node` object, we need to specify `left` and `right`;
`distance` is optional.




```
These three expressions correspond to the same distance matrix.
```
-  `nclusters` (padrão: `2` ) \ O número de clusters $ k $.
-  `npass` (padrão: `1` ) \ O número de vezes que o algoritmo de agrupamento $ k $ -medoids é realizada, cada vez com uma condição inicial diferente (aleatória). E se `initialid` é fornecido, o valor de `npass` é ignorado, pois o o algoritmo de agrupamento se comporta de forma determinística nesse caso.
-  `initialid` (padrão: `None` ) \ Especifica o clustering inicial a ser usado para o algoritmo EM. E se `initialid==None` , então um agrupamento inicial aleatório diferente é usado para cada uma das execuções `npass` do algoritmo EM. E se `initialid` não é `None` , então deve ser igual a uma matriz 1D contendo o número do cluster (entre `0` e `nclusters-1` ) para cada item. Cada cluster deve conter pelo menos um item. Com o clustering inicial especificado, o algoritmo EM é determinístico.

 Esta função retorna uma tupla `(clusterid, error, nfound)` , onde

 `clusterid` é uma matriz contendo o número do cluster ao qual

 cada item foi atribuído, o `error` é a soma das distâncias dentro do cluster

 para a solução de cluster $ k $ -medoids ideal, e `nfound` é o

 número de vezes que a solução ótima foi encontrada. Observe que o cluster

 número no `clusterid` é definido como o número do item do item

 representando o centróide do cluster.

 Agrupamento hierárquico

 Métodos de agrupamento hierárquico são inerentemente diferentes do

 $ k $ - significa método de agrupamento. No agrupamento hierárquico, a similaridade

 no perfil de expressão entre genes ou condições experimentais são

 representado na forma de uma estrutura de árvore. Esta estrutura de árvore pode ser

 mostrado graficamente por programas como Treeview e Java Treeview, que

 tem contribuído para a popularidade do agrupamento hierárquico no

 análise de dados de expressão gênica.

 A primeira etapa do agrupamento hierárquico é calcular a distância

 matriz, especificando todas as distâncias entre os itens a serem agrupados.

 Em seguida, criamos um nó unindo os dois itens mais próximos. Subseqüente

 nós são criados pela junção de itens ou nós com base no

 distância entre eles, até que todos os itens pertençam ao mesmo nó. Uma árvore

 estrutura pode então ser criada reconstituindo quais itens e nós foram

 fundido. Ao contrário do algoritmo EM, que é usado em $ k $ -means clustering,

 o processo completo de agrupamento hierárquico é determinístico.

 Existem vários tipos de clustering hierárquico, que diferem em como

 a distância entre os subnós é definida em termos de seus membros. Dentro

 `Bio.Cluster` , `Bio.Cluster` par a par, máxima, média e centróide

 Estão disponíveis.
-  No clustering de ligação única par, a distância entre dois nós é definido como a distância mais curta entre os pares distâncias entre os membros dos dois nós.
-  No agrupamento de ligação máxima em pares, alternativamente conhecido como clustering de ligação completa em pares, a distância entre dois nós é definido como a distância mais longa entre as distâncias de pares entre os membros dos dois nós.
-  No agrupamento de ligação média pareada, a distância entre dois nós é definido como a média de todas as distâncias de pares entre os itens dos dois nós.
-  No agrupamento de ligação centróide par, a distância entre dois nós é definido como a distância entre seus centróides. o os centróides são calculados tomando a média de todos os itens em um cluster. À medida que a distância de cada nó recém-formado até o existente nós e itens precisam ser calculados em cada etapa, a computação o tempo de agrupamento de ligação centróide par a par pode ser significativamente mais do que para os outros métodos de agrupamento hierárquico. Outro peculiaridade é que (para uma medida de distância baseada no Pearson correlação), as distâncias não aumentam necessariamente ao ir na árvore de agrupamento e pode até diminuir. Isso é causado por uma inconsistência entre o cálculo do centróide e a distância cálculo ao usar a correlação de Pearson: Considerando que o Pearson correlação efetivamente normaliza os dados para a distância cálculo, tal normalização não ocorre para o cálculo do centróide.

 Para agrupamento de ligação única, completa e média, o

 distância entre dois nós pode ser encontrada diretamente a partir das distâncias

 entre os itens individuais. Portanto, o algoritmo de agrupamento faz

 não precisa de acesso aos dados de expressão do gene original, uma vez que a distância

 matriz é conhecida. Para agrupamento de ligação centróide par a par, no entanto, o

 centróides de subnós recém-formados só podem ser calculados a partir do

 dados originais e não da matriz de distância.

 A implementação de agrupamento hierárquico de ligação única par a par é

 baseado no algoritmo SLINK (R. Sibson, 1973), que é muito mais rápido e

 mais eficiente em termos de memória do que uma implementação direta de pares

 clustering de ligação única. O resultado do agrupamento produzido por este

 algoritmo é idêntico à solução de agrupamento encontrada pelo

 algoritmo convencional de ligação única. A hierarquia de vínculo único

 algoritmo de agrupamento implementado nesta biblioteca pode ser usado para agrupar

 grandes conjuntos de dados de expressão de genes, para os quais

 algoritmos de clusterização falham devido a requisitos excessivos de memória e

 tempo de execução.

###  Representando uma solução de clustering hierárquico {# representar-a-hierarchical-clustering-solution .unnumbered}

 O resultado do agrupamento hierárquico consiste em uma árvore de nós, em

 em que cada nó junta dois itens ou subnós. Normalmente, não somos apenas

 interessado em quais itens ou subnós são unidos em cada nó, mas também

 em sua semelhança (ou distância) à medida que são unidos. Para armazenar um nó

 na árvore de agrupamento hierárquico, fazemos uso da classe `Node` ,

 que é definido em `Bio.Cluster` . Uma instância de `Node` tem três

 atributos:
-  `left`
-  `right`
-  `distance`

 Aqui, `left` e `right` são números inteiros que se referem aos dois itens ou

 subnós que são unidos neste nó, e a `distance` é a distância

 entre eles. Os itens sendo agrupados são numerados de 0 a

 $ \ left (\ textrm {number of items} - 1 \ right) $, enquanto os clusters são numerados

 de -1 a $ - \ left (\ textrm {número de itens} -1 \ right) $. Observe que o

 o número de nós é um a menos que o número de itens.

 Para criar um novo objeto `Node` , precisamos especificar `left` e `right` ;

 `distance` é opcional.


In [6]:
from Bio.Cluster import Node
Node(2, 3)

(2, 3): 0

In [7]:
Node(2, 3, 0.91)

(2, 3): 0.91


The attributes `left`, `right`, and `distance` of an existing `Node`
object can be modified directly:




Os atributos `left` , `right` e `distance` de um `Node` existente

 objeto pode ser modificado diretamente:


In [8]:
node = Node(4, 5)
node.left = 6
node.right = 2
node.distance = 0.73
node

(6, 2): 0.73


An error is raised if `left` and `right` are not integers, or if
`distance` cannot be converted to a floating-point value.

The Python class `Tree` represents a full hierarchical clustering
solution. A `Tree` object can be created from a list of `Node` objects:




Um erro é gerado se a `left` e a `right` não forem inteiros, ou se

 `distance` não pode ser convertida em um valor de ponto flutuante.

 A classe Python `Tree` representa um clustering hierárquico completo

 solução. Um objeto `Tree` pode ser criado a partir de uma lista de objetos `Node` :


In [9]:
from Bio.Cluster import Node, Tree
nodes = [Node(1, 2, 0.2), Node(0, 3, 0.5), Node(-2, 4, 0.6), Node(-1, -3, 0.9)]
tree = Tree(nodes)
print(tree)

(1, 2): 0.2
(0, 3): 0.5
(-2, 4): 0.6
(-1, -3): 0.9



The `Tree` initializer checks if the list of nodes is a valid
hierarchical clustering result:




O inicializador de `Tree` verifica se a lista de nós é válida

 resultado de agrupamento hierárquico:


In [11]:
nodes = [Node(1, 2, 0.2), Node(0, 2, 0.5)]
try:
    Tree(nodes)
    raise Exception("Should not arrive here")
except ValueError:
    print("This tree is problematic")

This tree is problematic



Individual nodes in a `Tree` object can be accessed using square
brackets:




Nós individuais em um objeto `Tree` podem ser acessados usando um quadrado

 colchetes:


In [12]:
nodes = [Node(1, 2, 0.2), Node(0, -1, 0.5)]
tree = Tree(nodes)
tree[0]

(1, 2): 0.2

In [13]:
tree[1]

(0, -1): 0.5

In [14]:
tree[-1]

(0, -1): 0.5


As a `Tree` object is read-only, we cannot change individual nodes in a
`Tree` object. However, we can convert the tree to a list of nodes,
modify this list, and create a new tree from this list:




Como um objeto `Tree` é somente leitura, não podemos alterar nós individuais em um

 Objeto de `Tree` . No entanto, podemos converter a árvore em uma lista de nós,

 modifique esta lista e crie uma nova árvore a partir desta lista:


In [15]:
tree = Tree([Node(1, 2, 0.1), Node(0, -1, 0.5), Node(-2, 3, 0.9)])
print(tree)

(1, 2): 0.1
(0, -1): 0.5
(-2, 3): 0.9


In [16]:
nodes = tree[:]
nodes[0] = Node(0, 1, 0.2)
nodes[1].left = 2
tree = Tree(nodes)
print(tree)

TypeError: sequence index must be integer, not 'slice'


This guarantees that any `Tree` object is always well-formed.

To display a hierarchical clustering solution with visualization
programs such as Java Treeview, it is better to scale all node distances
such that they are between zero and one. This can be accomplished by
calling the `scale` method on an existing `Tree` object:




Isso garante que qualquer objeto `Tree` esteja sempre bem formado.

 Para exibir uma solução de clustering hierárquico com visualização

 programas como Java Treeview, é melhor dimensionar todas as distâncias dos nós

 de forma que eles estão entre zero e um. Isso pode ser feito por

 chamando o método `scale` em um objeto `Tree` existente:


In [12]:
tree.scale()


This method takes no arguments, and returns `None`.

After hierarchical clustering, the items can be grouped into $k$
clusters based on the tree structure stored in the `Tree` object by
cutting the tree:




Este método não aceita argumentos e retorna `None` .

 Após o agrupamento hierárquico, os itens podem ser agrupados em $ k $

 clusters baseados na estrutura de árvore armazenada no objeto `Tree` por

 cortando a árvore:


In [13]:
clusterid = tree.cut(nclusters=1)

TypeError: cut() takes no keyword arguments


where `nclusters` (defaulting to `1`) is the desired number of clusters
$k$. This method ignores the top $k-1$ linking events in the tree
structure, resulting in $k$ separated clusters of items. The number of
clusters $k$ should be positive, and less than or equal to the number of
items. This method returns an array `clusterid` containing the number of
the cluster to which each item is assigned.

### Performing hierarchical clustering {#performing-hierarchical-clustering .unnumbered}

To perform hierarchical clustering, use the `treecluster` function in
`Bio.Cluster`.




onde `nclusters` (padronizado para `1` ) é o número desejado de clusters

 $ k $. Este método ignora os principais eventos de ligação $ k-1 $ na árvore

 estrutura, resultando em grupos de itens separados por $ k $. O número de

 os clusters $ k $ devem ser positivos e menores ou iguais ao número de

 Itens. Este método retorna uma matriz `clusterid` contendo o número de

 o cluster ao qual cada item é atribuído.

###  Executando clustering hierárquico {# perform-hierarchical-clustering .unnumbered}

 Para realizar o clustering hierárquico, use a função `treecluster` em

 `Bio.Cluster` .


In [14]:
from Bio.Cluster import treecluster
tree = treecluster(data)

NameError: name 'data' is not defined


where the following arguments are defined:

-   `data`\
    Array containing the data for the items.

-   `mask` (default: `None`)\
    Array of integers showing which data are missing. If `mask[i,j]==0`,
    then `data[i,j]` is missing. If `mask==None`, then all data
    are present.

-   `weight` (default: `None`)\
    The weights to be used when calculating distances. If
    `weight==None`, then equal weights are assumed.

-   `transpose` (default: `0`)\
    Determines if rows (`transpose==0`) or columns (`transpose==1`) are
    to be clustered.

-   `method` (default: `'m'`)\
    defines the linkage method to be used:

    -   `method=='s'`: pairwise single-linkage clustering

    -   `method=='m'`: pairwise maximum- (or complete-) linkage
        clustering

    -   `method=='c'`: pairwise centroid-linkage clustering

    -   `method=='a'`: pairwise average-linkage clustering

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]).

To apply hierarchical clustering on a precalculated distance matrix,
specify the `distancematrix` argument when calling `treecluster`
function instead of the `data` argument:




onde os seguintes argumentos são definidos:
-  `data` \ Matriz contendo os dados dos itens.
-  `mask` (padrão: `None` ) \ Matriz de inteiros mostrando quais dados estão faltando. Se `mask[i,j]==0` , então os `data[i,j]` estão faltando. Se `mask==None` , então todos os dados estão presentes.
-  `weight` (padrão: `None` ) \ Os pesos a serem usados ao calcular distâncias. E se `weight==None` , então pesos iguais são assumidos.
-  `transpose` (padrão: `0` ) \ Determina se as linhas ( `transpose==0` ) ou colunas ( `transpose==1` ) são a ser agrupado.
-  `method` (padrão: `&#39;m&#39;` ) \ define o método de ligação a ser usado:
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]).

 Para aplicar agrupamento hierárquico em uma matriz de distância pré-calculada,

 especifique o argumento `distancematrix` ao chamar o `treecluster`

 função em vez do argumento de `data` :


In [17]:
from Bio.Cluster import treecluster
tree = treecluster(distancematrix=distance)

NameError: name 'distance' is not defined


In this case, the following arguments are defined:

-   `distancematrix`\
    The distance matrix, which can be specified in three ways:

    -   as a 2D Numerical Python array (in which only the left-lower
        part of the array will be accessed):




Nesse caso, os seguintes argumentos são definidos:
-  `distancematrix` \ A matriz de distância, que pode ser especificada de três maneiras:


```
distance = array([[0.0, 1.1, 2.3],
                          [1.1, 0.0, 4.5],
                          [2.3, 4.5, 0.0]])

```


```
distance = array([[0.0, 1.1, 2.3], [1.1, 0.0, 4.5], [2.3, 4.5, 0.0]])
```



    -   as a 1D Numerical Python array containing consecutively the
        distances in the left-lower part of the distance matrix:




```
- as a 1D Numerical Python array containing consecutively the distances in the left-lower part of the distance matrix:
```


```
distance = array([1.1, 2.3, 4.5])

```


```

distance = array([1.1, 2.3, 4.5])

```



    -   as a list containing the rows of the left-lower part of the
        distance matrix:




```
- as a list containing the rows of the left-lower part of the distance matrix:
```


```
distance = [array([]),
                    array([1.1]),
                    array([2.3, 4.5])

```


```
distance = [array([]), array([1.1]), array([2.3, 4.5])
```



    These three expressions correspond to the same distance matrix. As
    `treecluster` may shuffle the values in the distance matrix as part
    of the clustering algorithm, be sure to save this array in a
    different variable before calling `treecluster` if you need
    it later.

-   `method`\
    The linkage method to be used:

    -   `method=='s'`: pairwise single-linkage clustering

    -   `method=='m'`: pairwise maximum- (or complete-) linkage
        clustering

    -   `method=='a'`: pairwise average-linkage clustering

    While pairwise single-, maximum-, and average-linkage clustering can
    be calculated from the distance matrix alone, pairwise
    centroid-linkage cannot.

When calling `treecluster`, either `data` or `distancematrix` should be
`None`.

This function returns a `Tree` object. This object contains
$\left(\textrm{number of items} - 1\right)$ nodes, where the number of
items is the number of rows if rows were clustered, or the number of
columns if columns were clustered. Each node describes a pairwise
linking event, where the node attributes `left` and `right` each contain
the number of one item or subnode, and `distance` the distance between
them. Items are numbered from 0 to
$\left(\textrm{number of items} - 1\right)$, while clusters are numbered
-1 to $-\left(\textrm{number of items}-1\right)$.

Self-Organizing Maps
--------------------

Self-Organizing Maps (SOMs) were invented by Kohonen to describe neural
networks (see for instance Kohonen, 1997 @kohonen1997). Tamayo (1999)
first applied Self-Organizing Maps to gene expression data @tamayo1999.

SOMs organize items into clusters that are situated in some topology.
Usually a rectangular topology is chosen. The clusters generated by SOMs
are such that neighboring clusters in the topology are more similar to
each other than clusters far from each other in the topology.

The first step to calculate a SOM is to randomly assign a data vector to
each cluster in the topology. If rows are being clustered, then the
number of elements in each data vector is equal to the number of
columns.

An SOM is then generated by taking rows one at a time, and finding which
cluster in the topology has the closest data vector. The data vector of
that cluster, as well as those of the neighboring clusters, are adjusted
using the data vector of the row under consideration. The adjustment is
given by
$$\Delta \underline{x}_{\textrm{cell}} = \tau \cdot \left(\underline{x}_{\textrm{row}} - \underline{x}_{\textrm{cell}} \right).$$
The parameter $\tau$ is a parameter that decreases at each iteration
step. We have used a simple linear function of the iteration step:
$$\tau = \tau_{\textrm{init}} \cdot \left(1 - {i \over n}\right),$$
$\tau_{\textrm{init}}$ is the initial value of $\tau$ as specified by
the user, $i$ is the number of the current iteration step, and $n$ is
the total number of iteration steps to be performed. While changes are
made rapidly in the beginning of the iteration, at the end of iteration
only small changes are made.

All clusters within a radius $R$ are adjusted to the gene under
consideration. This radius decreases as the calculation progresses as
$$R = R_{\textrm{max}} \cdot \left(1 - {i \over n}\right),$$ in which
the maximum radius is defined as
$$R_{\textrm{max}} = \sqrt{N_x^2 + N_y^2},$$ where
$\left(N_x, N_y\right)$ are the dimensions of the rectangle defining the
topology.

The function `somcluster` implements the complete algorithm to calculate
a Self-Organizing Map on a rectangular grid. First it initializes the
random number generator. The node data are then initialized using the
random number generator. The order in which genes or microarrays are
used to modify the SOM is also randomized. The total number of
iterations in the SOM algorithm is specified by the user.

To run `somcluster`, use




```
These three expressions correspond to the same distance matrix. As `treecluster` may shuffle the values in the distance matrix as part of the clustering algorithm, be sure to save this array in a different variable before calling `treecluster` if you need it later.
```
-  `method` \ O método de ligação a ser usado:

 Ao chamar o `treecluster` , os `data` ou a `treecluster` `distancematrix` devem ser

 `None` .

 Esta função retorna um objeto `Tree` . Este objeto contém

 $ \ left (\ textrm {number of items} - 1 \ right) $ nodes, onde o número de

 itens é o número de linhas se as linhas foram agrupadas ou o número de

 colunas se as colunas foram agrupadas. Cada nó descreve um par

 evento de ligação, onde os atributos de nó `left` e `right` contêm

 o número de um item ou subnó, e `distance` a distância entre

 eles. Os itens são numerados de 0 a

 $ \ left (\ textrm {number of items} - 1 \ right) $, enquanto os clusters são numerados

 -1 a $ - \ left (\ textrm {número de itens} -1 \ right) $.

 Mapas auto-organizáveis

 Mapas de auto-organização (SOMs) foram inventados por Kohonen para descrever

 redes (ver, por exemplo, Kohonen, 1997 @ kohonen1997). Tamayo (1999)

 aplicou pela primeira vez mapas auto-organizáveis a dados de expressão gênica @ tamayo1999.

 Os SOMs organizam itens em clusters que estão situados em alguma topologia.

 Normalmente, uma topologia retangular é escolhida. Os clusters gerados por SOMs

 são tais que os clusters vizinhos na topologia são mais semelhantes a

 uns aos outros do que clusters distantes uns dos outros na topologia.

 A primeira etapa para calcular um SOM é atribuir aleatoriamente um vetor de dados para

 cada cluster na topologia. Se as linhas estão sendo agrupadas, então o

 número de elementos em cada vetor de dados é igual ao número de

 colunas.

 Um SOM é então gerado pegando uma linha de cada vez e encontrando qual

 cluster na topologia tem o vetor de dados mais próximo. O vetor de dados de

 esse cluster, bem como os dos clusters vizinhos, são ajustados

 usando o vetor de dados da linha em consideração. O ajuste é

 dado por

 $$ \ Delta \ underline {x} *{\ textrm {cell}} = \ tau \ cdot \ left (\ underline {x}* {\ textrm {row}} - \ underline {x} _ {\ textrm {cell}} \ certo). $$

 O parâmetro $ \ tau $ é um parâmetro que diminui a cada iteração

 degrau. Usamos uma função linear simples da etapa de iteração:

 $$ \ tau = \ tau _ {\ textrm {init}} \ cdot \ left (1 - {i \ over n} \ right), $$

 $ \ tau _ {\ textrm {init}} $ é o valor inicial de $ \ tau $ conforme especificado por

 o usuário, $ i $ é o número da etapa de iteração atual e $ n $ é

 o número total de etapas de iteração a serem realizadas. Enquanto as mudanças são

 feito rapidamente no início da iteração, no final da iteração

 apenas pequenas mudanças são feitas.

 Todos os clusters em um raio $ R $ são ajustados ao gene sob

 consideração. Este raio diminui à medida que o cálculo avança conforme

 $$ R = R _ {\ textrm {max}} \ cdot \ left (1 - {i \ over n} \ right), $$ em que

 o raio máximo é definido como

 $$ R _ {\ textrm {max}} = \ sqrt {N_x ^ 2 + N_y ^ 2}, $$ onde

 $ \ left (N_x, N_y \ right) $ são as dimensões do retângulo que definem o

 topologia.

 A função `somcluster` implementa o algoritmo completo para calcular

 um mapa auto-organizado em uma grade retangular. Primeiro, ele inicializa o

 gerador de números aleatórios. Os dados do nó são então inicializados usando o

 gerador de números aleatórios. A ordem em que os genes ou microarrays são

 usado para modificar o SOM também é randomizado. O número total de

 iterações no algoritmo SOM são especificadas pelo usuário.

 Para executar `somcluster` , use


In [19]:
from Bio.Cluster import somcluster
clusterid, celldata = somcluster(data)

NameError: name 'data' is not defined


where the following arguments are defined:

-   `data` (required)\
    Array containing the data for the items.

-   `mask` (default: `None`)\
    Array of integers showing which data are missing. If `mask[i,j]==0`,
    then `data[i,j]` is missing. If `mask==None`, then all data
    are present.

-   `weight` (default: `None`)\
    contains the weights to be used when calculating distances. If
    `weight==None`, then equal weights are assumed.

-   `transpose` (default: `0`)\
    Determines if rows (`transpose` is `0`) or columns (`transpose` is
    `1`) are to be clustered.

-   `nxgrid, nygrid` (default: `2, 1`)\
    The number of cells horizontally and vertically in the rectangular
    grid on which the Self-Organizing Map is calculated.

-   `inittau` (default: `0.02`)\
    The initial value for the parameter $\tau$ that is used in the
    SOM algorithm. The default value for `inittau` is 0.02, which was
    used in Michael Eisen’s Cluster/TreeView program.

-   `niter` (default: `1`)\
    The number of iterations to be performed.

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]).

This function returns the tuple `(clusterid, celldata)`:

-   `clusterid`:\
    An array with two columns, where the number of rows is equal to the
    number of items that were clustered. Each row contains the $x$ and
    $y$ coordinates of the cell in the rectangular SOM grid to which the
    item was assigned.

-   `celldata`:\
    An array with dimensions
    $\left(\verb|nxgrid|, \verb|nygrid|, \textrm{number of columns}\right)$
    if rows are being clustered, or
    $\left(\verb|nxgrid|, \verb|nygrid|, \textrm{number of rows}\right)$
    if columns are being clustered. Each element `[ix][iy]` of this
    array is a 1D vector containing the gene expression data for the
    centroid of the cluster in the grid cell with coordinates
    `[ix][iy]`.

Principal Component Analysis
----------------------------

Principal Component Analysis (PCA) is a widely used technique for
analyzing multivariate data. A practical example of applying Principal
Component Analysis to gene expression data is presented by Yeung and
Ruzzo (2001) @yeung2001.

In essence, PCA is a coordinate transformation in which each row in the
data matrix is written as a linear sum over basis vectors called
principal components, which are ordered and chosen such that each
maximally explains the remaining variance in the data vectors. For
example, an $n \times 3$ data matrix can be represented as an
ellipsoidal cloud of $n$ points in three dimensional space. The first
principal component is the longest axis of the ellipsoid, the second
principal component the second longest axis of the ellipsoid, and the
third principal component is the shortest axis. Each row in the data
matrix can be reconstructed as a suitable linear combination of the
principal components. However, in order to reduce the dimensionality of
the data, usually only the most important principal components are
retained. The remaining variance present in the data is then regarded as
unexplained variance.

The principal components can be found by calculating the eigenvectors of
the covariance matrix of the data. The corresponding eigenvalues
determine how much of the variance present in the data is explained by
each principal component.

Before applying principal component analysis, typically the mean is
subtracted from each column in the data matrix. In the example above,
this effectively centers the ellipsoidal cloud around its centroid in 3D
space, with the principal components describing the variation of points
in the ellipsoidal cloud with respect to their centroid.

The function `pca` below first uses the singular value decomposition to
calculate the eigenvalues and eigenvectors of the data matrix. The
singular value decomposition is implemented as a translation in C of the
Algol procedure `svd` @golub1971, which uses Householder
bidiagonalization and a variant of the QR algorithm. The principal
components, the coordinates of each data vector along the principal
components, and the eigenvalues corresponding to the principal
components are then evaluated and returned in decreasing order of the
magnitude of the eigenvalue. If data centering is desired, the mean
should be subtracted from each column in the data matrix before calling
the `pca` routine.

To apply Principal Component Analysis to a rectangular matrix `data`,
use




onde os seguintes argumentos são definidos:
-  `data` (obrigatório) \ Matriz contendo os dados dos itens.
-  `mask` (padrão: `None` ) \ Matriz de inteiros mostrando quais dados estão faltando. Se `mask[i,j]==0` , então os `data[i,j]` estão faltando. Se `mask==None` , então todos os dados estão presentes.
-  `weight` (padrão: `None` ) \ contém os pesos a serem usados no cálculo de distâncias. E se `weight==None` , então pesos iguais são assumidos.
-  `transpose` (padrão: `0` ) \ Determina se as linhas ( `transpose` é `0` ) ou colunas ( `transpose` é `1` ) devem ser agrupados.
-  `nxgrid, nygrid` (padrão: `2, 1` ) \ O número de células horizontalmente e verticalmente no formato retangular grade na qual o Mapa Auto-Organizável é calculado.
-  `inittau` (padrão: `0.02` ) \ O valor inicial para o parâmetro $ \ tau $ que é usado no Algoritmo SOM. O valor padrão para `inittau` é 0,02, que era usado no programa Cluster / TreeView de Michael Eisen.
-  `niter` (padrão: `1` ) \ O número de iterações a serem executadas.
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]).

 Esta função retorna a tupla `(clusterid, celldata)` :
-  `clusterid` : \ Uma matriz com duas colunas, onde o número de linhas é igual ao número de itens que foram agrupados. Cada linha contém $ x $ e $ y $ coordenadas da célula na grade retangular SOM para a qual o o item foi atribuído.
-  `celldata` : \ Uma matriz com dimensões $ \ left (\ verb | nxgrid |, \ verb | nygrid |, \ textrm {número de colunas} \ right) $ se as linhas estão sendo agrupadas, ou $ \ left (\ verb | nxgrid |, \ verb | nygrid |, \ textrm {número de linhas} \ right) $ se as colunas estão sendo agrupadas. Cada elemento `[ix][iy]` deste array é um vetor 1D contendo os dados de expressão gênica para o centróide do cluster na célula da grade com coordenadas `[ix][iy]` .

 Análise do componente principal

 Análise de Componentes Principais (PCA) é uma técnica amplamente usada para

 analisar dados multivariados. Um exemplo prático de aplicação do Principal

 A análise de componentes para dados de expressão gênica é apresentada por Yeung e

 Ruzzo (2001) @ yeung2001.

 Em essência, o PCA é uma transformação de coordenadas em que cada linha no

 matriz de dados é escrita como uma soma linear sobre vetores de base chamados

 componentes principais, que são ordenados e escolhidos de modo que cada

 explica ao máximo a variância restante nos vetores de dados. Para

 exemplo, uma matriz de dados $ n \ vezes 3 $ pode ser representada como um

 nuvem elipsoidal de $ n $ pontos no espaço tridimensional. O primeiro

 componente principal é o eixo mais longo do elipsóide, o segundo

 componente principal, o segundo eixo mais longo do elipsóide, e o

 o terceiro componente principal é o eixo mais curto. Cada linha nos dados

 matriz pode ser reconstruída como uma combinação linear adequada do

 componentes principais. No entanto, a fim de reduzir a dimensionalidade de

 os dados, geralmente apenas os componentes principais mais importantes são

 retida. A variação restante presente nos dados é então considerada como

 variação inexplicada.

 Os componentes principais podem ser encontrados calculando os vetores próprios de

 a matriz de covariância dos dados. Os autovalores correspondentes

 determinar quanto da variação presente nos dados é explicada por

 cada componente principal.

 Antes de aplicar a análise de componentes principais, normalmente a média é

 subtraído de cada coluna na matriz de dados. No exemplo acima,

 isso centraliza efetivamente a nuvem elipsoidal em torno de seu centróide em 3D

 espaço, com os componentes principais descrevendo a variação de pontos

 na nuvem elipsoidal em relação ao seu centróide.

 A função `pca` abaixo primeiro usa a decomposição de valor singular para

 calcular os autovalores e autovetores da matriz de dados. o

 decomposição de valor singular é implementada como uma tradução em C do

 Procedimento Algol `svd` @ golub1971, que usa Householder

 bidiagonalização e uma variante do algoritmo QR. O diretor da escola

 componentes, as coordenadas de cada vetor de dados ao longo do principal

 componentes, e os autovalores correspondentes ao principal

 os componentes são avaliados e devolvidos em ordem decrescente do

 magnitude do valor próprio. Se a centralização de dados for desejada, a média

 deve ser subtraído de cada coluna na matriz de dados antes de chamar

 a rotina `pca` .

 Para aplicar a Análise de Componente Principal a `data` matriz retangular,

 usar


In [17]:
from Bio.Cluster import pca
columnmean, coordinates, components, eigenvalues = pca(data)

NameError: name 'data' is not defined


This function returns a tuple
`columnmean, coordinates, components, eigenvalues`:

-   `columnmean`\
    Array containing the mean over each column in `data`.

-   `coordinates`\
    The coordinates of each row in `data` with respect to the
    principal components.

-   `components`\
    The principal components.

-   `eigenvalues`\
    The eigenvalues corresponding to each of the principal components.

The original matrix `data` can be recreated by calculating
`columnmean +  dot(coordinates, components)`.

Handling Cluster/TreeView-type files
------------------------------------

Cluster/TreeView are GUI-based codes for clustering gene expression
data. They were originally written by [Michael
Eisen](http://rana.lbl.gov) while at Stanford University. `Bio.Cluster`
contains functions for reading and writing data files that correspond to
the format specified for Cluster/TreeView. In particular, by saving a
clustering result in that format, TreeView can be used to visualize the
clustering results. We recommend using Alok Saldanha’s
<http://jtreeview.sourceforge.net/><span>Java TreeView program</span>,
which can display hierarchical as well as $k$-means clustering results.

An object of the class `Record` contains all information stored in a
Cluster/TreeView-type data file. To store the information contained in
the data file in a `Record` object, we first open the file and then read
it:




Esta função retorna uma tupla

 `columnmean, coordinates, components, eigenvalues` :
-  `columnmean` \ Matriz contendo a média sobre cada coluna de `data` .
-  `coordinates` \ As coordenadas de cada linha nos `data` em relação ao componentes principais.
-  `components` \ Os principais componentes.
-  `eigenvalues` \ Os autovalores correspondentes a cada um dos componentes principais.

 Os `data` matriz original podem ser recriados calculando

 `columnmean + dot(coordinates, components)` .

 Tratamento de arquivos do tipo Cluster / TreeView

 Cluster / TreeView são códigos baseados em GUI para expressão de genes de cluster

 dados. Eles foram originalmente escritos por [Michael

 Eisen] ( [http://rana.lbl.gov](http://rana.lbl.gov) ) enquanto estava na Stanford University. `Bio.Cluster`

 contém funções para ler e escrever arquivos de dados que correspondem a

 o formato especificado para Cluster / TreeView. Em particular, salvando um

 resultado de agrupamento nesse formato, TreeView pode ser usado para visualizar o

 resultados de agrupamento. Recomendamos o uso de Alok Saldanha&#39;s

 [http://jtreeview.sourceforge.net/](http://jtreeview.sourceforge.net/) &lt;span&gt; programa Java TreeView &lt;/span&gt;,

 que pode exibir tanto resultados hierárquicos quanto $ k $ -means agrupamento.

 Um objeto da classe `Record` contém todas as informações armazenadas em um

 Arquivo de dados do tipo Cluster / TreeView. Para armazenar as informações contidas em

 o arquivo de dados em um objeto `Record` , primeiro abrimos o arquivo e depois lemos

 isto:


In [20]:
from Bio import Cluster
handle = open("mydatafile.txt")
record = Cluster.read(handle)
handle.close()

FileNotFoundError: [Errno 2] No such file or directory: 'mydatafile.txt'


This two-step process gives you some flexibility in the source of the
data. For example, you can use




Este processo de duas etapas oferece alguma flexibilidade na origem do

 dados. Por exemplo, você pode usar


In [19]:
import gzip # Python standard library
handle = gzip.open("mydatafile.txt.gz")

FileNotFoundError: [Errno 2] No such file or directory: 'mydatafile.txt.gz'


to open a gzipped file, or




para abrir um arquivo compactado, ou


In [20]:
import urllib # Python standard library
handle = urllib.urlopen("http://somewhere.org/mydatafile.txt")


AttributeError: module 'urllib' has no attribute 'urlopen'


to open a file stored on the Internet before calling `read`.

The `read` command reads the tab-delimited text file `mydatafile.txt`
containing gene expression data in the format specified for Michael
Eisen’s Cluster/TreeView program. For a description of this file format,
see the manual to Cluster/TreeView. It is available at [Michael Eisen’s
lab website](http://rana.lbl.gov/manuals/ClusterTreeView.pdf) and at
[our
website](http://bonsai.ims.u-tokyo.ac.jp/~mdehoon/software/cluster/cluster3.pdf).

A `Record` object has the following attributes:

-   `data`\
    The data array containing the gene expression data. Genes are stored
    row-wise, while microarrays are stored column-wise.

-   `mask`\
    This array shows which elements in the `data` array, if any,
    are missing. If `mask[i,j]==0`, then `data[i,j]` is missing. If no
    data were found to be missing, `mask` is set to `None`.

-   `geneid`\
    This is a list containing a unique description for each gene (i.e.,
    ORF numbers).

-   `genename`\
    This is a list containing a description for each gene (i.e.,
    gene name). If not present in the data file, `genename` is set to
    `None`.

-   `gweight`\
    The weights that are to be used to calculate the distance in
    expression profile between genes. If not present in the data file,
    `gweight` is set to `None`.

-   `gorder`\
    The preferred order in which genes should be stored in an
    output file. If not present in the data file, `gorder` is set to
    `None`.

-   `expid`\
    This is a list containing a description of each microarray, e.g.
    experimental condition.

-   `eweight`\
    The weights that are to be used to calculate the distance in
    expression profile between microarrays. If not present in the data
    file, `eweight` is set to `None`.

-   `eorder`\
    The preferred order in which microarrays should be stored in an
    output file. If not present in the data file, `eorder` is set to
    `None`.

-   `uniqid`\
    The string that was used instead of UNIQID in the data file.

After loading a `Record` object, each of these attributes can be
accessed and modified directly. For example, the data can be
log-transformed by taking the logarithm of `record.data`.

### Calculating the distance matrix {#calculating-the-distance-matrix .unnumbered}

To calculate the distance matrix between the items stored in the record,
use




para abrir um arquivo armazenado na Internet antes de chamar `read` .

 O comando `read` lê o arquivo de texto delimitado por tabulação `mydatafile.txt`

 contendo dados de expressão gênica no formato especificado para Michael

 Programa Cluster / TreeView de Eisen. Para obter uma descrição deste formato de arquivo,

 consulte o manual para Cluster / TreeView. Está disponível em [Michael Eisen&#39;s

 site do laboratório] ( [http://rana.lbl.gov/manuals/ClusterTreeView.pdf](http://rana.lbl.gov/manuals/ClusterTreeView.pdf) ) e em

 [nosso

 website] ( [http://bonsai.ims.u-tokyo.ac.jp/~mdehoon/software/cluster/cluster3.pdf](http://bonsai.ims.u-tokyo.ac.jp/~mdehoon/software/cluster/cluster3.pdf) ).

 Um objeto `Record` possui os seguintes atributos:
-  `data` \ A matriz de dados contendo os dados de expressão do gene. Genes são armazenados em linha, enquanto os microarrays são armazenados em coluna.
-  `mask` \ Esta matriz mostra quais elementos da matriz de `data` , se houver, estão faltando. Se `mask[i,j]==0` , então os `data[i,j]` estão faltando. Se não dados foram encontrados em falta, a `mask` está definida como `None` .
-  `geneid` \ Esta é uma lista contendo uma descrição única para cada gene (ou seja, Números ORF).
-  `genename` \ Esta é uma lista contendo uma descrição para cada gene (ou seja, nome do gene). Se não estiver presente no arquivo de dados, `genename` é definido como `None` .
-  `gweight` \ Os pesos que devem ser usados para calcular a distância em perfil de expressão entre genes. Se não estiver presente no arquivo de dados, `gweight` é definido como `None` .
-  `gorder` \ A ordem preferida em que os genes devem ser armazenados em um arquivo de saída. Se não estiver presente no arquivo de dados, `gorder` é definido como `None` .
-  `expid` \ Esta é uma lista contendo uma descrição de cada microarray, por exemplo condição experimental.
-  `eweight` \ Os pesos que devem ser usados para calcular a distância em perfil de expressão entre microarrays. Se não estiver presente nos dados arquivo, `eweight` é definido como `None` .
-  `eorder` \ A ordem preferida em que os microarrays devem ser armazenados em um arquivo de saída. Se não estiver presente no arquivo de dados, o `eorder` é definido como `None` .
-  `uniqid` \ A string que foi usada em vez de UNIQID no arquivo de dados.

 Depois de carregar um objeto `Record` , cada um desses atributos pode ser

 acessado e modificado diretamente. Por exemplo, os dados podem ser

 transformado em log usando o logaritmo de `record.data` .

###  Calculando a matriz de distância {# calculating-the-distance-matrix .unnumbered}

 Para calcular a matriz de distância entre os itens armazenados no registro,

 usar


In [21]:
matrix = record.distancematrix()

NameError: name 'record' is not defined


where the following arguments are defined:

-   `transpose` (default: `0`)\
    Determines if the distances between the rows of `data` are to be
    calculated (`transpose==0`), or between the columns of `data`
    (`transpose==1`).

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]).

This function returns the distance matrix as a list of rows, where the
number of columns of each row is equal to the row number (see section
\[subsec:distancematrix\]).

### Calculating the cluster centroids {#calculating-the-cluster-centroids .unnumbered}

To calculate the centroids of clusters of items stored in the record,
use




onde os seguintes argumentos são definidos:
-  `transpose` (padrão: `0` ) \ Determina se as distâncias entre as linhas de `data` devem ser calculado ( `transpose==0` ), ou entre as colunas de `data` ( `transpose==1` ).
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]).

 Esta função retorna a matriz de distância como uma lista de linhas, onde o

 o número de colunas de cada linha é igual ao número da linha (ver seção

 [subsec: distancematrix]).

###  Calculando os centróides do cluster {# calculating-the-cluster-centroids .unnumbered}

 Para calcular os centróides de grupos de itens armazenados no registro,

 usar


In [22]:
cdata, cmask = record.clustercentroids()

NameError: name 'record' is not defined


-   `clusterid` (default: `None`)\
    Vector of integers showing to which cluster each item belongs. If
    `clusterid` is not given, then all items are assumed to belong to
    the same cluster.

-   `method` (default: `'a'`)\
    Specifies whether the arithmetic mean (`method=='a'`) or the median
    (`method=='m'`) is used to calculate the cluster center.

-   `transpose` (default: `0`)\
    Determines if the centroids of the rows of `data` are to be
    calculated (`transpose==0`), or the centroids of the columns of
    `data` (`transpose==1`).

This function returns the tuple `cdata, cmask`; see section
\[subsec:clustercentroids\] for a description.

### Calculating the distance between clusters {#calculating-the-distance-between-clusters-1 .unnumbered}

To calculate the distance between clusters of items stored in the
record, use



- `clusterid` (padrão: `None` ) \ Vetor de inteiros mostrando a qual cluster cada item pertence. E se `clusterid` não é fornecido, então todos os itens são considerados pertencentes a o mesmo cluster.
-  `method` (padrão: `&#39;a&#39;` ) \ Especifica se a média aritmética ( `method==&#39;a&#39;` ) ou a mediana ( `method==&#39;m&#39;` ) é usado para calcular o centro do cluster.
-  `transpose` (padrão: `0` ) \ Determina se os centróides das linhas de `data` devem ser calculado ( `transpose==0` ), ou os centróides das colunas de `data` ( `transpose==1` ).

 Esta função retorna a tupla `cdata, cmask` ; veja a seção

 [subsec: clustercentroids] para uma descrição.

###  Calculando a distância entre clusters {# calculating-the-distance-between-clusters-1 .unnumbered}

 Para calcular a distância entre os grupos de itens armazenados no

 registro, uso


In [23]:
distance = record.clusterdistance()

NameError: name 'record' is not defined


where the following arguments are defined:

-   `index1` (default: `0`)\
    A list containing the indices of the items belonging to the
    first cluster. A cluster containing only one item $i$ can be
    represented either as a list `[i]`, or as an integer `i`.

-   `index2` (default: `0`)\
    A list containing the indices of the items belonging to the
    second cluster. A cluster containing only one item $i$ can be
    represented either as a list `[i]`, or as an integer `i`.

-   `method` (default: `'a'`)\
    Specifies how the distance between clusters is defined:

    -   `'a'`: Distance between the two cluster centroids (arithmetic
        mean);

    -   `'m'`: Distance between the two cluster centroids (median);

    -   `'s'`: Shortest pairwise distance between items in the two
        clusters;

    -   `'x'`: Longest pairwise distance between items in the two
        clusters;

    -   `'v'`: Average over the pairwise distances between items in the
        two clusters.

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]).

-   `transpose` (default: `0`)\
    If `transpose==0`, calculate the distance between the rows of
    `data`. If `transpose==1`, calculate the distance between the
    columns of `data`.

### Performing hierarchical clustering {#performing-hierarchical-clustering-1 .unnumbered}

To perform hierarchical clustering on the items stored in the record,
use




onde os seguintes argumentos são definidos:
-  `index1` (padrão: `0` ) \ Uma lista contendo os índices dos itens pertencentes ao primeiro cluster. Um cluster contendo apenas um item $ i $ pode ser representado como uma lista `[i]` ou como um inteiro `i` .
-  `index2` (padrão: `0` ) \ Uma lista contendo os índices dos itens pertencentes ao segundo cluster. Um cluster contendo apenas um item $ i $ pode ser representado como uma lista `[i]` ou como um inteiro `i` .
-  `method` (padrão: `&#39;a&#39;` ) \ Especifica como a distância entre os clusters é definida:
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]).
-  `transpose` (padrão: `0` ) \ Se `transpose==0` , calcule a distância entre as linhas de `data` . Se `transpose==1` , calcule a distância entre os colunas de `data` .

###  Executando clustering hierárquico {# perform-hierarchical-clustering-1 .unnumbered}

 Para realizar clustering hierárquico nos itens armazenados no registro,

 usar


In [24]:
tree = record.treecluster()

NameError: name 'record' is not defined


where the following arguments are defined:

-   `transpose` (default: `0`)\
    Determines if rows (`transpose==0`) or columns (`transpose==1`) are
    to be clustered.

-   `method` (default: `'m'`)\
    defines the linkage method to be used:

    -   `method=='s'`: pairwise single-linkage clustering

    -   `method=='m'`: pairwise maximum- (or complete-) linkage
        clustering

    -   `method=='c'`: pairwise centroid-linkage clustering

    -   `method=='a'`: pairwise average-linkage clustering

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]).

-   `transpose`\
    Determines if genes or microarrays are being clustered. If
    `transpose==0`, genes (rows) are being clustered. If `transpose==1`,
    microarrays (columns) are clustered.

This function returns a `Tree` object. This object contains
$\left(\textrm{number of items} - 1\right)$ nodes, where the number of
items is the number of rows if rows were clustered, or the number of
columns if columns were clustered. Each node describes a pairwise
linking event, where the node attributes `left` and `right` each contain
the number of one item or subnode, and `distance` the distance between
them. Items are numbered from 0 to
$\left(\textrm{number of items} - 1\right)$, while clusters are numbered
-1 to $-\left(\textrm{number of items}-1\right)$.

### Performing $k$-means or $k$-medians clustering {#performing-k-means-or-k-medians-clustering .unnumbered}

To perform $k$-means or $k$-medians clustering on the items stored in
the record, use




onde os seguintes argumentos são definidos:
-  `transpose` (padrão: `0` ) \ Determina se as linhas ( `transpose==0` ) ou colunas ( `transpose==1` ) são a ser agrupado.
-  `method` (padrão: `&#39;m&#39;` ) \ define o método de ligação a ser usado:
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]).
-  `transpose` \ Determina se genes ou microarrays estão sendo agrupados. E se `transpose==0` , genes (linhas) estão sendo agrupados. Se `transpose==1` , microarrays (colunas) são agrupados.

 Esta função retorna um objeto `Tree` . Este objeto contém

 $ \ left (\ textrm {number of items} - 1 \ right) $ nodes, onde o número de

 itens é o número de linhas se as linhas foram agrupadas ou o número de

 colunas se as colunas foram agrupadas. Cada nó descreve um par

 evento de ligação, onde os atributos de nó `left` e `right` contêm

 o número de um item ou subnó, e `distance` a distância entre

 eles. Os itens são numerados de 0 a

 $ \ left (\ textrm {number of items} - 1 \ right) $, enquanto os clusters são numerados

 -1 a $ - \ left (\ textrm {número de itens} -1 \ right) $.

###  Executando $ k $ -means ou $ k $ -medians clustering {# performing-k-means-or-k-medians-clustering .unnumbered}

 Para realizar $ k $ -means ou $ k $ -medians agrupamento nos itens armazenados em

 o registro, use


In [25]:
clusterid, error, nfound = record.kcluster()


NameError: name 'record' is not defined


where the following arguments are defined:

-   `nclusters` (default: `2`)\
    The number of clusters $k$.

-   `transpose` (default: `0`)\
    Determines if rows (`transpose` is `0`) or columns (`transpose` is
    `1`) are to be clustered.

-   `npass` (default: `1`)\
    The number of times the $k$-means/-medians clustering algorithm is
    performed, each time with a different (random) initial condition. If
    `initialid` is given, the value of `npass` is ignored and the
    clustering algorithm is run only once, as it behaves
    deterministically in that case.

-   `method` (default: `a`)\
    describes how the center of a cluster is found:

    -   `method=='a'`: arithmetic mean ($k$-means clustering);

    -   `method=='m'`: median ($k$-medians clustering).

    For other values of `method`, the arithmetic mean is used.

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]).

This function returns a tuple `(clusterid, error, nfound)`, where
`clusterid` is an integer array containing the number of the cluster to
which each row or cluster was assigned, `error` is the within-cluster
sum of distances for the optimal clustering solution, and `nfound` is
the number of times this optimal solution was found.

### Calculating a Self-Organizing Map {#calculating-a-self-organizing-map .unnumbered}

To calculate a Self-Organizing Map of the items stored in the record,
use




onde os seguintes argumentos são definidos:
-  `nclusters` (padrão: `2` ) \ O número de clusters $ k $.
-  `transpose` (padrão: `0` ) \ Determina se as linhas ( `transpose` é `0` ) ou colunas ( `transpose` é `1` ) devem ser agrupados.
-  `npass` (padrão: `1` ) \ O número de vezes que o algoritmo de agrupamento $ k $ -means / -medians é realizada, cada vez com uma condição inicial diferente (aleatória). E se `initialid` é fornecido, o valor de `npass` é ignorado e o algoritmo de agrupamento é executado apenas uma vez, como se comporta deterministicamente nesse caso.
-  `method` (padrão: `a` ) \ descreve como o centro de um cluster é encontrado:
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]).

 Esta função retorna uma tupla `(clusterid, error, nfound)` , onde

 `clusterid` é uma matriz inteira contendo o número do cluster para

 a qual cada linha ou cluster foi atribuído, o `error` é o dentro do cluster

 soma das distâncias para a solução de cluster ideal, e `nfound` é

 o número de vezes que essa solução ótima foi encontrada.

###  Calculating a Self-Organizing Map {# calculating-a-self-organizing-map .unnumbered}

 Para calcular um Mapa de Auto-Organização dos itens armazenados no registro,

 usar


In [26]:
clusterid, celldata = record.somcluster()


NameError: name 'record' is not defined


where the following arguments are defined:

-   `transpose` (default: `0`)\
    Determines if rows (`transpose` is `0`) or columns (`transpose` is
    `1`) are to be clustered.

-   `nxgrid, nygrid` (default: `2, 1`)\
    The number of cells horizontally and vertically in the rectangular
    grid on which the Self-Organizing Map is calculated.

-   `inittau` (default: `0.02`)\
    The initial value for the parameter $\tau$ that is used in the
    SOM algorithm. The default value for `inittau` is 0.02, which was
    used in Michael Eisen’s Cluster/TreeView program.

-   `niter` (default: `1`)\
    The number of iterations to be performed.

-   `dist` (default: `'e'`, Euclidean distance)\
    Defines the distance function to be used
    (see \[sec:distancefunctions\]).

This function returns the tuple `(clusterid, celldata)`:

-   `clusterid`:\
    An array with two columns, where the number of rows is equal to the
    number of items that were clustered. Each row contains the $x$ and
    $y$ coordinates of the cell in the rectangular SOM grid to which the
    item was assigned.

-   `celldata`:\
    An array with dimensions
    $\left(\verb|nxgrid|, \verb|nygrid|, \textrm{number of columns}\right)$
    if rows are being clustered, or
    $\left(\verb|nxgrid|, \verb|nygrid|, \textrm{number of rows}\right)$
    if columns are being clustered. Each element `[ix][iy]` of this
    array is a 1D vector containing the gene expression data for the
    centroid of the cluster in the grid cell with coordinates
    `[ix][iy]`.

### Saving the clustering result {#saving-the-clustering-result .unnumbered}

To save the clustering result, use




onde os seguintes argumentos são definidos:
-  `transpose` (padrão: `0` ) \ Determina se as linhas ( `transpose` é `0` ) ou colunas ( `transpose` é `1` ) devem ser agrupados.
-  `nxgrid, nygrid` (padrão: `2, 1` ) \ O número de células horizontalmente e verticalmente no formato retangular grade na qual o Mapa Auto-Organizável é calculado.
-  `inittau` (padrão: `0.02` ) \ O valor inicial para o parâmetro $ \ tau $ que é usado no Algoritmo SOM. O valor padrão para `inittau` é 0,02, que era usado no programa Cluster / TreeView de Michael Eisen.
-  `niter` (padrão: `1` ) \ O número de iterações a serem executadas.
-  `dist` (padrão: `&#39;e&#39;` , distância euclidiana) \ Define a função de distância a ser usada (consulte [sec: funções distância]).

 Esta função retorna a tupla `(clusterid, celldata)` :
-  `clusterid` : \ Uma matriz com duas colunas, onde o número de linhas é igual ao número de itens que foram agrupados. Cada linha contém $ x $ e $ y $ coordenadas da célula na grade retangular SOM para a qual o o item foi atribuído.
-  `celldata` : \ Uma matriz com dimensões $ \ left (\ verb | nxgrid |, \ verb | nygrid |, \ textrm {número de colunas} \ right) $ se as linhas estão sendo agrupadas, ou $ \ left (\ verb | nxgrid |, \ verb | nygrid |, \ textrm {número de linhas} \ right) $ se as colunas estão sendo agrupadas. Cada elemento `[ix][iy]` deste array é um vetor 1D contendo os dados de expressão gênica para o centróide do cluster na célula da grade com coordenadas `[ix][iy]` .

###  Salvando o resultado do clustering {# saving-the-clustering-result .unnumbered}

 Para salvar o resultado do agrupamento, use


In [27]:
record.save(jobname, geneclusters, expclusters)


NameError: name 'record' is not defined


where the following arguments are defined:

-   `jobname`\
    The string `jobname` is used as the base name for names of the files
    that are to be saved.

-   `geneclusters`\
    This argument describes the gene (row-wise) clustering result. In
    case of $k$-means clustering, this is a 1D array containing the
    number of the cluster each gene belongs to. It can be calculated
    using `kcluster`. In case of hierarchical clustering, `geneclusters`
    is a `Tree` object.

-   `expclusters`\
    This argument describes the (column-wise) clustering result for the
    experimental conditions. In case of $k$-means clustering, this is a
    1D array containing the number of the cluster each experimental
    condition belongs to. It can be calculated using `kcluster`. In case
    of hierarchical clustering, `expclusters` is a `Tree` object.

This method writes the text file `jobname.cdt`, `jobname.gtr`,
`jobname.atr`, `jobname*.kgg`, and/or `jobname*.kag` for subsequent
reading by the Java TreeView program. If `geneclusters` and
`expclusters` are both `None`, this method only writes the text file
`jobname.cdt`; this file can subsequently be read into a new `Record`
object.

Example calculation
-------------------

This is an example of a hierarchical clustering calculation, using
single linkage clustering for genes and maximum linkage clustering for
experimental conditions. As the Euclidean distance is being used for
gene clustering, it is necessary to scale the node distances `genetree`
such that they are all between zero and one. This is needed for the Java
TreeView code to display the tree diagram correctly. To cluster the
experimental conditions, the uncentered correlation is being used. No
scaling is needed in this case, as the distances in `exptree` are
already between zero and two. The example data `cyano.txt` can be found
in the `data` subdirectory.




onde os seguintes argumentos são definidos:
-  `jobname` \ A string `jobname` é usada como o nome base para os nomes dos arquivos que devem ser salvos.
-  `geneclusters` \ Este argumento descreve o resultado do agrupamento de genes (por linha). Dentro caso de $ k $ -means clustering, este é um array 1D contendo o número do cluster ao qual cada gene pertence. Pode ser calculado usando `kcluster` . Em caso de clustering hierárquico, `geneclusters` é um objeto `Tree` .
-  `expclusters` \ Este argumento descreve o resultado do agrupamento (em colunas) para o condições experimentais. No caso de $ k $ -means agrupamento, este é um Array 1D contendo o número do cluster de cada experimental condição pertence. Pode ser calculado usando `kcluster` . Em caso de agrupamento hierárquico, `expclusters` é um objeto `Tree` .

 Este método grava o arquivo de texto `jobname.cdt` , `jobname.gtr` ,

 `jobname.atr` , `jobname*.kgg` e / ou `jobname*.kag` para os

 leitura pelo programa Java TreeView. Se `geneclusters` e

 `expclusters` são ambos `None` , este método grava apenas o arquivo de texto

 `jobname.cdt` ; este arquivo pode ser posteriormente lido em um novo `Record`

 objeto.

 Cálculo de exemplo

 Este é um exemplo de cálculo de agrupamento hierárquico, usando

 agrupamento de ligação única para genes e agrupamento de ligação máxima para

 condições experimentais. Como a distância euclidiana está sendo usada para

 agrupamento de genes, é necessário escalonar as distâncias dos nós `genetree`

 de forma que todos eles estão entre zero e um. Isso é necessário para o Java

 Código TreeView para exibir o diagrama de árvore corretamente. Para agrupar o

 condições experimentais, a correlação não centrada está sendo usada. Não

 a escala é necessária neste caso, pois as distâncias em `exptree` são

 já entre zero e dois. O exemplo de dados `cyano.txt` pode ser encontrado

 no subdiretório de `data` .


In [28]:
from Bio import Cluster
handle = open("cyano.txt")
record = Cluster.read(handle)
handle.close()
genetree = record.treecluster(method='s')
genetree.scale()
exptree = record.treecluster(dist='u', transpose=1)
record.save("cyano_result", genetree, exptree)


FileNotFoundError: [Errno 2] No such file or directory: 'cyano.txt'


This will create the files `cyano_result.cdt`, `cyano_result.gtr`, and
`cyano_result.atr`.

Similarly, we can save a $k$-means clustering solution:




Isso criará os arquivos `cyano_result.cdt` , `cyano_result.gtr` e

 `cyano_result.atr` .

 Da mesma forma, podemos economizar uma solução de clustering $ k $ -means:


In [29]:
from Bio import Cluster
handle = open("cyano.txt")
record = Cluster.read(handle)
handle.close()
(geneclusters, error, ifound) = record.kcluster(nclusters=5, npass=1000)
(expclusters, error, ifound) = record.kcluster(nclusters=2, npass=100, transpose=1)
record.save("cyano_result", geneclusters, expclusters)


FileNotFoundError: [Errno 2] No such file or directory: 'cyano.txt'